<a href="https://colab.research.google.com/github/a-massive-world/music_video/blob/main/planet_music_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output as clear
import json,time,os
os.chdir('/content/')
ifile = 'installed_libraries.json'

try:
  with open(ifile,'r') as f:
    i_file = json.load(f)

    print('I already have the necessary libraries. ^-^\n')
except FileNotFoundError:
  print('I am installing libraries\n')

  ##################################
  !python -m pip uninstall Pillow -y
  !pip install Pillow==8.2.0
  !pip install spotipy --upgrade
  !pip install qrcode[pil]
  !pip install fonttools
  !pip install humanfriendly
  !pip install py-deezer
  !pip install pydub
  !pip install -U ipython
  !pip install lyricsgenius
  # !pip install IPython --upgrade
  ##################################

  with open(ifile,'w') as f:
    json.dump('1',f)

  time.sleep(2)
  clear()

  os.kill(os.getpid(),9)



In [ ]:
from PIL import ImageDraw, Image, ImageFont, ImageOps
from pydeezer import Deezer, Downloader
from pydeezer.constants import track_formats
from pydeezer.exceptions import LoginError
from IPython.display import Audio as ad, Image as im, clear_output as clear,Video
from datetime import date as datetimedate, datetime, timedelta
from bs4 import BeautifulSoup as bsp
from humanfriendly import format_timespan, format_size
from urllib.parse import quote
from fontTools.ttLib import TTFont as ttfont
from fontTools.unicode import Unicode
from mutagen.mp3 import MP3
from pydub import AudioSegment
from contextlib import redirect_stderr
from google.colab import files
from fontTools import ttLib
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
from spotipy.exceptions import SpotifyException
import random, qrcode, re, lyricsgenius, requests, math,pytz, spotipy
import string as st, json, time, textwrap, os,shutil
import unicodedata
from matplotlib import pyplot as plt
import scipy.cluster,sklearn.cluster
from matplotlib.ticker import FormatStrFormatter
import matplotlib


empty_list = ['',""," ",' ']
allowed_punct = ["(",")","?","!","’","‘","“","”",".","¿",","]
non_space_punct = ["?","!",",",";","-",'.',"."]
sa = ['NotoSerifDevanagari-Regular.ttf','NotoSerifBengali-Regular.ttf']
left_side = ["‘","“","(","¿"]
right_side = ["”","’",")"]
lst = ['&\u200b','Featuring\u200b','Released\u200b', 'on\u200b','Produced\u200b', 'by\u200b','On\u200b']

##### THE FOLDERS 'ALL_FONTS', 'unlimited_ssm', 'My_Music' SHOULD BE IN DRIVE

separator = ' —— '
sep = ' — '
fd = 'font_database'
da = 'deezer_appendix'
lyrics_sync = ''
slsd = 'song_lyrics_sync_dictionary'
dmd = 'downloaded_music_database'
sid = 'song_info_database'
default = 'NotoSans-Regular.ttf'
font_path = '/content/drive/MyDrive/essentials/ALL_FONTS/'
dynamic_range = [0,255]
color = (154,154,154)
placeholder_images = '/content/placeholder_images/'
td = 'translation_database'
video_path = '/content/placeholder_videos/'
archive = '/content/drive/MyDrive/essentials/unlimited_ssm/---archive---/'
key_destination = '/content/drive/MyDrive/essentials/unlimited_ssm/---archive---/video/'
p1 ='\'!"(),*+.:%“;<!¬~=>?[\\]^_`।{|}।~،؟¿¡”（）‘’？！'
p2 ='\*+:;<।¬~=>[\\]?!^_`।{|}~،؟¿¡？！'
m = '/content/drive/MyDrive/essentials/ALL_FONTS/merged.ttf'
n = '/content/drive/MyDrive/essentials/ALL_FONTS/'
number_of_requests = 10000
maximum_allowed_character_length = 100
time_to_sleep = 1


if os.path.exists(placeholder_images) is False:
  os.mkdir(placeholder_images)

if os.path.exists(video_path) is False:
  os.mkdir(video_path)

if os.path.exists('/content/SYNC_MUSIC/') is False:
  os.mkdir('/content/SYNC_MUSIC/')

if os.path.exists('/content/sample_data/') is True:
  for files in os.listdir('/content/sample_data/'):
    os.remove('/content/sample_data/'+files)
  os.rmdir('/content/sample_data')

In [ ]:

def get_sd():
 return to_json({},'song_details','read')

def to_json(dic, name, string):
  # print('to_json:',string,sep,name)c
  path = '/content/drive/MyDrive/essentials/unlimited_ssm/---archive---/jsons/'
  a = 1
  if string == 'write':
    
    if dic == {}:
      print('Given Dictionary is empty. Check again!!')
      return False
    if a == 1:  

      with open(path+name+'.json','w') as fp:
        json.dump(dic,fp)
    else:
      print('Did not save anything.')
      return False
    
    # print('saved '+name+'.json')
    print('saved',sep,name,'('+getfilesize(path+name+'.json',1)+')')

  if string == 'read':
    with open(path+name+'.json','r') as fp:
      data = json.load(fp)
    
    return data


def getfilesize(the_file,the_type):
  file_size = os.path.getsize(the_file)
  actual_file_size = format_size(file_size)

  la_output = the_file.split('/')[-1]+sep+actual_file_size
  
  if the_type not in empty_list:
    if the_type == 1:
      return la_output.split(sep)[-1]
    else:
      return file_size
  else:
    print(la_output)


def genius_search(genius_link):
  s = re.sub("-"," ",genius_link.split('/')[-1].split('lyrics')[0]).strip()
  return s

def zero_width_cleaner(s,is_this_lyrics):
  zw = ["\u200b","\xa0","\u2005","\u2006","\u2026","\u3000","\u300b"]
  zws = ["\u2005","\u2006","\u300b","\u3000","\u2026","･･･"]
  if is_this_lyrics not in empty_list:
    # p1 ='\'!"(),*+.:%“;<!¬~=>?[\\]^_`।{|}।~،؟¿¡”（）‘’？！'
    # s = s.translate(str.maketrans(" "," ",p1))
    # print(s)
    try:
      s = s.replace("！","! ")
    except Exception as e:
      clear()
      print(s)
      print('Song index: ',index)

      print('Run planet_translate, google forsaken us!!! The lyrics one, perhaps.')

  for word in zw:
    if word in zws:
      # s = re.sub(word," ",s)
      s = s.replace(word," ")
    else:
      s = re.sub(word,"",s)  
  
  regex = r"([0-9].[0-9]+[a-zA-Z]|[0-9]|[0-9][a-zA-Z])+(EmbedShare|Embed)| URLCopyEmbedCopy|EmbedShare|Embed" #https://regex101.com/r/WmB5iV/2
  s= re.sub(regex,"",s,0,re.MULTILINE)
  # s = re.sub(r"[0-9]+EmbedShare URLCopyEmbedCopy","",s)                              
  # # s = re.sub(r"([0-9]\.[0-9]+[a-zA-Z]|[0-9]|[0-9][a-zA-Z])+Embed|\Embed","",s)
  # s = re.sub(r"[0-9]+Embed","",s)
  # s = re.sub(r"EmbedShare URLCopyEmbedCopy","",s)
  # s = re.sub(r"EmbedShare","",s)
  # s = re.sub(r"Embed","",s)
  

  s = re.sub(r"[\[].*?[]]","",s)
  
  a= re.sub("（","(",s)
  a=re.sub("）",")",a)
  a = re.sub("？","?",a)
  # if is_this_lyrics == 1:  
  a= re.sub(r"\s[\"\'\‘\’\”\“](?!\w)|^[\']\s","",a) #### kills rogue apostrophes #https://regex101.com/r/TyfvO3/2


  a = re.sub(r"\”(?!\w)","”",a)
  a = re.sub(r"\'(?!\w)","’",a)
  a = re.sub(r"\"(?!\w)","”",a)
  a = re.sub(r"′","’",a)

  a = re.sub(r"(?<!\w)\"","“",a)
  # a = re.sub(r"[^\S]?(\(.*?\))[^\S]?", r" \1 ", a).strip() ## puts space before parenthesis

  a = re.sub('\'','’',a)
  # a = re.sub('.','',a)
  a = re.sub('\"','“',a)

  a = re.sub(r"(?<!\w)\’",'‘',a)
  
  # a = re.sub(r"(?<!\w)\”","“",a)v

  s = a
  return s

try:
  if timercircuit == 1:
    timercircuit = 1
except NameError:
  to_json((0,0,int(datetime.utcnow().timestamp())),'timer_circuit','write')
  timercircuit = 1

In [ ]:
zero_width_cleaner('꿈 속에서 봤던 déjà vu',1)

In [ ]:
def fragile_translator_machine(word):
    # word = 'Suerte que en sur hayas nachido'
  # print('Translating',sep,word)
  chart = len(word.split()[0::2])
  if chart == 1:
    chart=2
  
  chart = 3
  # print('sleep time',sep,chart)
  # url = "https://clients5.google.com/translate_a/single?client=gtx&sl=auto&tl=en&dt=rm&q=" + quote(word)
  url = 'https://translate.googleapis.com/translate_a/single?client=gtx&sl=auto&tl=en&dt=t&dt=rm&q='+quote(word)+'&ie=UTF-8&oe=UTF-8'
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'
  }
  requests.session().close()
  
  # time.sleep(chart)
  trans = requests.get(url,headers=headers)
  requests.session().close()
      
  try:
      trans.json()
  except json.decoder.JSONDecodeError:
      print('ERRRRRRRRRRRRRRROR')
      # os.kill(os.getpid(),9)
      return False, False
  try:
      pronounciation = trans.json()[0][1][-1]
  except IndexError:
      pronounciation = trans.json()[0][0][1]
  translation = trans.json()[0][0][0]
  main = trans.json()[0][0][1]

  # pronounciation,translation
  if main==pronounciation:
      # clear()
      return translation,word
  else:
      # clear()
      return translation,pronounciation
  
def translation_bouncer(line):
  # line = 'Bunny is a rider'
  i,character_length,last_execution_time = to_json({},'timer_circuit','read')
  character_length+=len(line)
  i+=1
  
  
  # current_time = int(datetime.utcnow().timestamp())
  # delayed = current_time-last_execution_time
  if character_length>maximum_allowed_character_length and character_length!=0 or i %number_of_requests == 0 and i!=0:
    if int(datetime.utcnow().timestamp())-last_execution_time<100:
      time.sleep(time_to_sleep)
      if character_length>maximum_allowed_character_length:
        character_length=0

  j,k = fragile_translator_machine(line)
  # current_time = int(datetime.utcnow().timestamp())
  to_json((i,character_length,int(datetime.utcnow().timestamp())),'timer_circuit','write')
    # print('\n')
    # print(i)
  if (j,k) == (False,False):
    print("THE END IS HEREEEEEEEEEE!!! HALP :(")
    return False,False

  sk = to_json({},'translation_database','read')
  # sk={}
  sk[line]={'main':line,
            'pronounciation':k,
            'translation':j}
  to_json(sk,'translation_database','write')        
  return sk[line]


def translating(line):

  sk = to_json({},'translation_database','read')
  

  line = re.sub('‘','\'',line)
  line = re.sub('“','"',line)
  line = re.sub('”','"',line)
  line = re.sub('’','\'',line)
  # sk={}
  # sk = normalize_keys(s)
  # print('THE LINE IS',line)
  try:
    j,k = sk[line]['translation'],sk[line]['pronounciation']
    if j in ['\u200b','\u200b\u200b']:
      return '\u200b','\u200b'
    return re.sub("[\s\.,]+(?=$)","",zero_width_cleaner(j,1)),re.sub("[\s\.,]+(?=$)","",zero_width_cleaner(k,1))
  except KeyError:
    print('Translating',sep,line)
    plop = translation_bouncer(line)
    clear()
    return re.sub("[\s\.,]+(?=$)","",zero_width_cleaner(plop['translation'],1)),re.sub("[\s\.,]+(?=$)","",zero_width_cleaner(plop['pronounciation'],1))

In [ ]:
def normalize_keys(dictionary):
  lst = list(dictionary.keys())
  for item in lst:
    # print(item)
    new_key = norm(str(item))
    dictionary[new_key]=dictionary.pop(item)

  return dictionary
def norm(a):
  a = a.split()
  
  w = bytes('',encoding='utf-8')
  for word in a:
    my_unicode = word
    my_unicode = my_unicode.strip('/')
    my_unicode = my_unicode.replace('/','')
    output = unicodedata.normalize('NFD', my_unicode).encode('ascii', 'ignore')
    if len(output) != len(word):
      word_b = unicodedata.normalize('NFD',word).encode('utf-8','ignore')
      if b'\xd0\xb5' in word_b:
        output = word_b.replace(b'\xd0\xb5',b'e')

    if output in [b'']:
      w+=bytes(word+' ',encoding='utf-8')
    else:
      w+=output+bytes(' ',encoding='utf-8')

  
  return w.strip()



In [ ]:
####### SPOTIFY TOKEN COLLECTOR

def deezer_arl_checker():
  deezer_arl = to_json({},'deezer_arl','read')
  try:
    deezer = Deezer(arl = deezer_arl)
  except LoginError:
    print('Current arl: '+deezer_arl+'\n')
    deezer_arl = input('Get Deezer arl from https://www.deezer.com/en/\nF12 --> APPLICATION --> COOKIES --> deezer.com\n\ndeezer_arl: ')
    to_json(deezer_arl,'deezer_arl','write')
    deezer = Deezer(arl = deezer_arl)

  user_info = deezer.user
  return deezer


def spotify_token_checker():
  

  url = 'https://developer.spotify.com/console/get-playlist-tracks/?playlist_id=&market=&fields=&limit=&offset=&additional_types='

  t = to_json({},'spotify_ids','read')
  spotify = spotipy.Spotify(auth=t['token'],
                            auth_manager = SpotifyClientCredentials(
                                client_id = t['client_id'],
                                client_secret = t['client_secret']
                            ))


  try:
    teste = spotify.user_playlist_tracks(playlist_id=t['playlist'],
                                        limit=1,
                                        offset=0)
  except SpotifyException:
    clear()
    time_difference(timenow(''),t['last_update'])
    t['last_update']=timenow('')
    print('get the token--> ',url)
    time.sleep(1)
    token = input('\n\nAccess token from spotify.com: ')
    
    t['token']=token
    to_json(t,'spotify_ids','write')
    spotify = spotipy.Spotify(auth=token,
                          auth_manager = SpotifyClientCredentials(
                              client_id = t['client_id'],
                              client_secret = t['client_secret']
                          ))


  return spotify

def soundcloud_token_checker():
  CLIENT_ID  = to_json({},'soundcloud_client_id','read')
  api = SoundcloudAPI(CLIENT_ID)
  try:
    api.resolve('https://soundcloud.com/user-232440117/using-you')
  except Exception as e:
    clear()
    print('Go to this https://soundcloud.com/discover\n')
    CLIENT_ID = input('F12 --> Network --> filter out "client_id" and paste the first link here.\n')
    if CLIENT_ID not in empty_list:
      CLIENT_ID = CLIENT_ID.split('client_id=')[-1].split('&app')[0]
    while CLIENT_ID in empty_list:
      print('Enter valid shit')
      CLIENT_ID = input('F12 --> Network --> filter out client_id and paste here.\n')

    CLIENT_ID = CLIENT_ID.split('client_id=')[-1].split('&app')[0]
    api = SoundcloudAPI(CLIENT_ID)
    api.resolve('https://soundcloud.com/user-232440117/using-you')
    to_json(CLIENT_ID,'soundcloud_client_id','write')
  
  return CLIENT_ID,api




In [ ]:
deezer_arl = to_json({},'deezer_arl','read')
deezer = deezer_arl_checker()
user_info = deezer.user
api_caller,GENIUS_ACCESS_TOKEN = to_json({},'xx','read')

try:
  if api_caller == 1:
    print('I already have the genius access token. :)\n')
except NameError:
  print('generate and get client access token\nhttps://genius.com/api-clients\n\n')
  GENIUS_ACCESS_TOKEN = input('Access token from Genius.com: ')
genius = lyricsgenius.Genius(GENIUS_ACCESS_TOKEN)
genius.skip_non_songs=True
api_caller= 1
to_json((api_caller,GENIUS_ACCESS_TOKEN),'xx','write')
time.sleep(2)
clear()

def test_line():
  the_line = ''

  while the_line in empty_list:
      
    s = to_json({},slsd,'read')
    song_keys = list(s.keys())
    rand_index = random.randint(0,len(song_keys))
    choosen_song_index = song_keys[rand_index]
    the_song  = s[str(choosen_song_index)]
    lines = the_song.split('\n')[0::2]
    choosen_line_index = random.randint(0,len(lines))
    try:
      the_line = lines[choosen_line_index].split(']')[-1]
    except IndexError:
      continue
  return the_line
  
trans_pis= translating(test_line())  
trans_pis

In [ ]:
len(to_json({},'translation_database','read'))

In [ ]:
from IPython.display import clear_output as clear
import json


ifile = 'installed_libraries-MANIM.json'
try:
  with open(ifile,'r') as f:
    i_file = json.load(f)
  
  print('I already have the necessary libraries. ^-^\n')
except FileNotFoundError:

  print('I am installing MANIM libraries\n')


  import time
  import os
  if trans_pis != (False,False) and deezer.arl == deezer_arl:

    !sudo apt install libcairo2-dev ffmpeg \
        texlive texlive-latex-extra texlive-fonts-extra \
        texlive-latex-recommended texlive-science \
        tipa libpango1.0-dev
    !sudo apt install libpango1.0-dev pkg-config python3-dev
    clear()
    !pip install manim==0.14.0
    !pip install googletrans==4.0.0-rc1
    

    # !pip install ffmpeg-python
    # !pip install imageio-ffmpeg
    !sudo pip install imageio==2.4.1
    !pip install imageio-ffmpeg
    # !pip install ManimPango
    !pip install webcolors
    !pip install ffmpeg-python

    clear()
    clear()
    time.sleep(2)
    with open(ifile,'w') as f:
      json.dump('1',f)

    os.kill(os.getpid(),9)
  
  else:
    clear()
    print('Factory Reset Time')

In [ ]:
######## LIBEND run above
# from manim import *
import manimpango

def timer_for_google_translate(line):
  j,k = (False,False)
  print('Translating',sep,line)
  sleeper = 2
  c = 0
  while (j,k)==(False,False):
    c+=1
    # if sleeper>2:
    #   sleeper = 1
    time.sleep(sleeper)
    sleeper=2
    print('try #'+str(c))
    if c%10==0:
      clear()


    j,k = translating(line)
  clear()
  return j,k


In [ ]:

def hexcolor(r,g,b):
  return '#%02x%02x%02x' % (r,g,b)
def font_name(font_path):
    font = ttLib.TTFont(font_path, ignoreDecompileErrors=True)
    with redirect_stderr(None):
        names = font['name'].names

    details = {}
    for x in names:
        if x.langID == 0 or x.langID == 1033:
            try:
                details[x.nameID] = x.toUnicode()
            except UnicodeDecodeError:
                details[x.nameID] = x.string.decode(errors='ignore')

    return details[4], details[1], details[2]

# def markup_texter(line,color_dictionary):

#   w = ''
#   for a in fontaine(line):
#     f = font_name(font_path+a['font'])[0].replace(' Regular','')
#     if f in ['Noto Sans']:
#       f = 'Arial Unicode MS'
          
#     for word in the_lists:
#       if color_dictionary not in empty_list:
#         try:
#           r,g,b = color_dictionary[strips_words(word.strip().lower(),0,1)]
#         except KeyError:
#           r,g,b = (154,154,154)
#       else:
#         r,g,b = (154,154,154)
      
#       c = hexcolor(r,g,b)
      
#       w+='<span font="'+f+'">'+'<span fgcolor="'+c+'">'+word+' '+'</span></span>'
#       # w+=
#   clear()
#   return w

# import manimpango
# # manimpango.list_fonts()

# manimpango.register_font(m)
# for files in regular_files:
#   manimpango.register_font(font_path+files)
def markup_texter(line,color_dictionary,is_this_lyrics):
  w = ''
  # line = line.replace('&\u200b','&\u200bamp;')
  # line = line.replace('&','&amp;')  
  if is_this_lyrics == 1:
    line = zero_width_cleaner(line,is_this_lyrics)
  low = fontaine(line)
  for i in range(len(low)):
    # print(low[i])
    a=  low[i]
    f = font_name(font_path+a['font'])[0].replace(' Regular','')
    if f in ['Noto Sans']:
      f = 'Arial Unicode MS'
    # print(f)
    the_lists=a['word'].split()
    
    if f in ['Noto Naskh Arabic']:
      # print('arabic')
      the_lists=a['word'].split()
    
    blow = the_lists          
    for j in range(len(the_lists)):
      word = the_lists[j]
      # display('wordthelist[j] '+word)
      # blow = the_lists
      if color_dictionary not in empty_list:
        try:
          r,g,b = color_dictionary[strips_words(word,0,1)]
        except KeyError:
          r,g,b = (154,154,154)
      else:
        r,g,b = (154,154,154)
      
      c = hexcolor(r,g,b)
      
      # print(r,g,b)
      
      # if i<(len(low)):
      w+='<span font="'+f+'">'+'<span fgcolor="'+c+'">'+word+'</span></span>'
      
      
      
      try:
        # display('BLOW: ',blow)
        if blow[j+1][0] not in non_space_punct and blow[j+1][0] not in right_side or is_this_lyrics in empty_list:
          if blow[j][-1] not in left_side or is_this_lyrics in empty_list:
            c = hexcolor(154,154,154)
            w+='<span font="'+'Arial Unicode MS'+'">'+'<span fgcolor="'+c+'">'+' '+'</span></span>'
      except IndexError:
        break
      
      
    try:
      if low[i+1]['word'][0] not in non_space_punct and low[i+1]['word'][0] not in right_side or is_this_lyrics in empty_list:
        if low[i]['word'][-1] not in left_side or is_this_lyrics in empty_list:

          c = hexcolor(154,154,154)
          w+='<span font="'+'Arial Unicode MS'+'">'+'<span fgcolor="'+c+'">'+' '+'</span></span>'
    except IndexError:
      break
    
      # w+='<span font="'+f+'">'+'<span fgcolor="'+c+'">'+word+'</span></span>'

      # if list_of_words[i+1]['word'][0] not in non_space_punct and list_of_words[i+1]['word'][0] not in right_side:
      #     if list_of_words[i]['word'][-1] not in left_side:      
      #       # img_array.append(blank_space)
      #       w+=' '

      # w+=
  
  # w = w.replace("&","&amp;")
  # display(w)
  return w

In [ ]:

def get_key():
  key = "".join(random.sample(st.ascii_lowercase+st.ascii_uppercase,16))
  return key


def check_if_keyed_in_database(num):

  d = to_json({},'downloaded_music_database','read')

  for key in d:
    if num == d[key]['file_index']:
      t = 1
      break
    else:
      t = 0
  return t


In [ ]:
def update_dl():
  music = '/content/drive/MyDrive/essentials/My_Music/'
  # done_dl = to_json({},dmd,'read')
  done_dl = {}
  done_dl_keys = done_dl.keys()
  count = 0
  for folders in os.listdir(music):
    for files in os.listdir(music+folders):
      if files.endswith('.mp3'):
        count +=1
        # key = get_key()
        num,name = files.split('}')
        
        num = int(num.strip('{').strip())
        name = name.split('.mp3')[0].strip()
        loc = music+folders+'/'+files
        
        # if count%10==0:
        #   clear()

        if str(num) in done_dl_keys:
          continue
        # print(num,name)
        
        # print(count+1,num,name,'added')
        
        done_dl[num] = {'file_index': num,
                        'location':loc,
                        'file_name':name}

  to_json(done_dl,dmd,'write')

  return to_json({},dmd,'read')




In [ ]:
from webcolors import name_to_rgb

def modify_color(color):
    # if any value is less than 63, make it 63
    color = tuple(max(63, c) for c in color)

    # find the minimum and maximum values
    min_val = min(color)
    max_val = max(color)
    # print('modified',color)

    # check if the difference between the minimum and maximum values is less than 27
    # if max_val - min_val < 27:
    #     # find the index of the non-maximum value
    #     idx = color.index(min_val) if color[0] != max_val else color.index(min(color[1:], default=color[1]))
    #     # modify the non-maximum value
    #     color = list(color)
    #     color[idx] = max(63, min(color[idx] + 27, 255))
    #     color = tuple(color)

    return color


def translate_and_check_color(word):
    # Translate word to English
    # translator = Translator()
    # translated_word = translator.translate(word, dest='en')
    translated_word = translating(word)[0].lower().replace("the ","")
    if translated_word.lower() in ['white','black','gray','grey']:
      return None
    # print(translated_word)
    else:
    # Check if translated word is a color name
    
      try:
        r,g,b = name_to_rgb(translated_word.lower())
        color_rgb = (r,g,b)
        print(translated_word,sep,color_rgb)
        return modify_color(color_rgb)
      except Exception as e:
        color_rgb = None


    
        return color_rgb



def ran_col():
  a=b=c=0
  
  d = 0+63
  e = 255
  thres = int((e-d)/7)
  while abs(a-b)<thres and abs(a-c)<thres and abs(b-c)<thres:
    a,b,c = random.randint(d,e),random.randint(d,e),random.randint(d,e)
  # print(a,b,c)
  return a,b,c

def colors(string):
  
  color_dictionary = {}
  s = strips_words(string,0,1)
  # s = norm(s).decode('utf-8')
  c= {}
  for words in s.split():
    if words not in empty_list:
      # rgb = translate_and_check_color(words)
      rgb = None

      if rgb is None:
        color_dictionary[ran_col()]=words
      else:
        color_dictionary[rgb]=words
  for item in color_dictionary:
    c[color_dictionary[item]]=item

    
    
    
  return c

# text = "আমার জামা হলুদ না, সাদা। এবং আমার জুতা নীল, তবে আমার কমলা চশমা আছে। আমার বাগানে সবুজ ঘাস আছে এবং আমার বাসায় কালো ফার্নিচার রয়েছে। আমার মা প্রতিদিন লাল জামা পরে। আমি কখনও বেগুনি রং পছন্দ করি না।"
# text = "সাদা লাল সবুজ কালো নীল কমলা বেগুনি"
# text = "বেগুনি নীল আকাশী সবুজ হলুদ কমলা লাল"
# text = 'morado azul gris Verde amarillo naranja rojo'

# dic = colors(text)
# color_frequency_bubble(text,colors(text))
# display(dic)

In [ ]:

def color_frequency_bubble(lyrics,color_dictionary):
  import collections
  s = strips_words(lyrics,1,1)
  counts = collections.Counter(s)
  new_list = sorted(s, key=counts.get, reverse=True)
  sorted_list = list(dict.fromkeys(new_list))
  if dd != []:
    news = sorted_list[0:len(dd)]
    kort = 0
    for iter in news:
      # print(iter,kort,dd[kort])
      if dd[kort] not in empty_list:
        color_dictionary[iter]=dd[kort]
      kort+=1



  # t = [str(s.count(x))+'\u2006'+x for x in set(s)]
  t1 = {x:s.count(x) for x in set(s)}
  gg1 = ''
  a = re.sub('\n\n\n','\u200b','\n'.join((sorted_list)))
  # a = re.sub('\n\n\n','\u200b',lyrics)
  
  a = re.sub('\n\n','\u200b',a)
  a = re.sub('\n',' ',a).strip()
  a = re.sub('\u200b','\n',a)

  gt= a

  s = re.sub(r"[\u2026]"," ", gt) # strips the ellipses

  gt = s
  captions= s

  stripped_lines = []
  count = total_character = 0
  for lines in captions.split():
    if len(lines)!=0 and lines !=' ' and lines != '':
      count+=1
      total_character += len(lines.strip())

  brrt = int(np.ceil(total_character/31))
  if brrt>80:
    brrt = int(np.ceil(total_character/40))
  if brrt<10:
    brrt = int(np.ceil(total_character/20))




  list_of_lines = limiting_line_space(captions,brrt)
  g01= VGroup()
  count= 0
  for line in list_of_lines:
    g00 = VGroup()
    for word in line.split():
      # print(word)
      if word in empty_list:
        continue
      # a = MarkupText(markup_texter(word.split('\u2006')[-1].strip(),color_dictionary,1),font_size=35).scale(t1[strips_words(word.split('\u2006')[-1].strip(),0,1)])
      
      a = MarkupText(markup_texter(word.split('\u2006')[-1].strip(),color_dictionary,1),font_size=35).scale(abs(len(sorted_list)-count*2))
    
      # a = MarkupText(markup_texter(word.split('\u2006')[-1].strip(),color_dictionary,1),font_size=35)
      count+=1
      clear()
      g00+=a
    g00.arrange(RIGHT, buff=1)
    g01 += g00
  g01.arrange(DOWN,buff=0.07)
  box = Rectangle(  # create a box
            height=config.frame_height, width=config.frame_width, fill_color=hexcolor(255,255,100), 
            fill_opacity=0.5, stroke_color=hexcolor(255,255,100)
        )
  g01.scale_to_fit_height(config.frame_height*0.98)
  if g01.width > config.frame_width:
    g01.scale_to_fit_width(config.frame_width*.98)
  background = ImageMobject(Image.new('RGB',[res,res],(56,56,56))).scale_to_fit_width(config.frame_width)
  os.chdir('/content/')
  print('SORTED BY MOST FREQUENT WORD TO LEAST FREQUENT')
  class lyr(Scene):
    def construct(self):
      self.add(background,g01)

  %manim -qh -r $res,$res -v WARNING lyr


In [ ]:
def see_color_palette(pixel,dic):
  
  res = 512
  i=0
  a = pixel
  pixel = int(2**pixel)
  im = Image.new('RGBA',[res,res],(0,0,0,0))
  com = int((res*res)/(pixel*pixel))
  if dic in empty_list:
    
    print('Colors:',com)
    margin=offset=0
    for i in range(0,com):
      image = Image.new('RGBA',[pixel,pixel],ran_col())
      im.paste(image,(margin,offset))
      if margin>=res-pixel:
        offset+=pixel
        margin=0
        continue
      margin +=pixel
    return im
  else:
    print('Dictionary color:',len(dic))
    if len(dic)>com:
      p = pixel
      while len(dic)>com:
        pixel = int(2**a)
        com = int((res**2)/(pixel**2))
        a-=1
      print('pixel size of',p,'is too big to show',len(dic),'colors\nChanging pixel to',pixel)
    
    margin = offset = 0
    for i in dic:
      image = Image.new('RGBA',[pixel,pixel],dic[i])
      im.paste(image,(margin,offset))
      if margin>=res-pixel:
        offset+=pixel
        margin=0
        continue
      margin +=pixel
    if margin!= 0:
      im = im.crop([0,0,res,offset+pixel])
    else:
      im = im.crop([0,0,res,offset])
  return im,dic


In [ ]:
def play_audio(music):
  index = ''
  done_dl = update_dl()
  sd = get_sd()
  clear()
  lk = []
  while lk==[]:
    word = input('The song contains? ')
    if word not in empty_list:
      for item in sd:
        if norm(word.lower()) in norm(sd[item]['file_name'].lower()):
          lk.append(sd[item]['file_name']+separator+"{"+item+"}")
    else:
      lk=[]
  lk = sorted(lk)
  c= 0 
  print('\n')
  for item in lk:
    c+=1
    print(str(c)+'. '+item)

  choice = input('\nChoose index of song: [x: ignore, ENTER: first one]\n')
  if choice in ['x','X']:
    clear()
    return 'ok bai ;P'
  else:
    if choice in empty_list:
      clear()
      choice = 0
      index = lk[choice].split(separator)[-1].strip('{}')
      
      if music in empty_list:
        print('Playing '+done_dl[index]['file_name']+'....'+'\n'+'Index:',index+'\n')
        time.sleep(2)
        return ad(done_dl[index]['location'],embed=True,autoplay=True)
      else:
        print('Showing data for '+done_dl[index]['file_name']+'....')
        return done_dl[index]

    else:
      clear()
      choice = int(choice)-1
      index = lk[choice].split(separator)[-1].strip('{}')
      if music in empty_list:
        print('Playing '+done_dl[index]['file_name']+'....'+'\n'+'Index:',index+'\n')
        time.sleep(2)
        return ad(done_dl[index]['location'],embed=True,autoplay=True)
      else:
        print('Showing data for '+done_dl[index]['file_name']+'....')
        return done_dl[index]


In [ ]:

def num_to_ordinal(p):
  n = int(p)
  if 4 <= n % 100 <= 20:
    a = str(n) + 'th'
  else: 
    a = str(n) + {1: 'st', 2: 'nd', 3: 'rd'}.get(n%10,'th')
  return a

def get_elements(lists):
  f = ''
  for i in range(len(lists)):
    f += zero_width_cleaner(lists[i]['name'],'')+', '
    if i== len(lists)-2:
      f = f.strip(', ')
      f += ' &\u200b '

  return " "+f.strip(', ')



In [ ]:
def starting_point():
  # if os.path.exists('/content/media/'):
  #   shutil.rmtree('/content/media/')


  # done_dl = update_dl()
  sd = get_sd()
  clear()
  lk = []
  while lk==[]:
    word = input('The song contains? ')
    for item in sd:
      if norm(word.lower()) in norm(sd[item]['file_name'].lower()):
        lk.append(sd[item]['file_name']+separator+"{"+item+"}")
  lk = sorted(lk)
  c= 0 
  print('\n')
  for item in lk:
    c+=1
    print(str(c)+'. '+item)

  choice = input('\nChoose index of song: [ENTER: first one]\n')
  
  if choice in empty_list:
    choice = int(lk[0].split(separator)[-1].strip('{}'))

  else:
    choice = int(choice.strip())
    choice = int(lk[choice-1].split(separator)[-1].strip('{}'))

  choice = str(choice)
  
  links = sd[choice]['link'].keys()
  # id = link.split('/')[-1]

  clear()
  # print(choice)
  print(choice, sd[choice]['file_name'],'choosen')


  dmd_c  = done_dl[str(choice)]

  song_name = dmd_c['file_name']
  file_index = dmd_c['file_index']
  loc = dmd_c['location']
  index = str(file_index)

  print(song_name,separator,file_index)
  if 'deezer' in links:
    track = deezer.get_track(int(sd[choice]['link']['deezer'].split('/')[-1]))
    print('Lyrics: ','LYRICS' in track['info'].keys())
    

    if 'LYRICS' in track['info'].keys():
      try:
        lyrics_sync = track['info']['LYRICS']['LYRICS_SYNC_JSON']
      except KeyError:
        lyrics_sync = ''
    else:
      lyrics_sync = ''
  else:
    print('Lyrics: ',False)
    track = lyrics_sync= ''
  print('Sync data:',lyrics_sync not in empty_list)
  print('\n')
  return index,lyrics_sync,track

In [ ]:
def get_deezer_track_data(index):
  track = deezer.get_track(int(get_sd()[index]['link']['deezer'].split('/')[-1]))
  display(get_sd()[index]) 
  print('\n') 
  print('Lyrics:','LYRICS' in track['info'])
  if 'LYRICS' in track['info']:
    print('Lyrics sync:','LYRICS_SYNC_JSON' in track['info']['LYRICS'])
  return track
def genius_to_lyrics(index,split_lyrics,override):
  
  song = to_json({},sid,'read')
  list_of_song = song.keys()

  if index in list_of_song and override in empty_list:
    url  = song[index]['url']
    print(song[index]['song_name']+' already saved\n')
    # if override not in empty_list:
    return song[index]['lyrics']
  else:
    try:
      a,b = done_dl[index]['file_name'].replace('by\u200b','by').split(' by ')
      # b = re.sub(",","",b)
    except ValueError:
      a=done_dl[index]['file_name']
      b=''
    search_url = "https://genius.com/search?q="+quote(a+' '+b.split(',')[0])
    print('CLICKKKKKK',search_url)
    print('\n\n')
    if desparate_time in empty_list:
      url = input('Enter link for '+done_dl[index]['file_name']+'\n')
    else:
      url = ''
    
    try:
      
      enter_song_data_from_genius_link(url,index,split_lyrics)
    except Exception as e:
      clear()
      # if desparate_time == 1:
      #   deezer_song_data_for_desparate_times()
      #   return '\nCollecting data from Deezer cause genius.com is being a bitch...\n\n'
      # else:
      print('Gotta do things manually cause genius.com is being a bitch\n')
      print('Win+R and type cmd --> cd desktop --> python song.py\n')
      print('Copy this link:')
      display(url)
      print('\n')
      print('Upload singular_song_data.json from:')
      display('C:\\Users\\SHAJID\\Desktop')
      print('\n')
      the_file = files.upload()
      json_path='/content/drive/MyDrive/essentials/unlimited_ssm/---archive---/jsons/'
      shutil.copy('singular_song_data.json',json_path+'singular_song_data.json')
      time.sleep(1)
      os.remove('singular_song_data.json')
      dat = to_json({},'singular_song_data','read')
      s = to_json({},sid,'read')
      s[index]=dat
      s[index]['mp3_file_index']=index
      s[index]['lyrics']=''.join(dat['lyrics'].split('You might also like\n'))
      s[index]['lyrics']='\n'.join(s[index]['lyrics'].split('You might also like'))
      
      if split_lyrics == 1:
        lyrics = s[index]['lyrics']
        print('splitting hairs')
        # display(lyrics)
        lyrics = bird_cage(lyrics,'')
        s[index]['lyrics']=lyrics
      to_json(s,sid,'write')

      # print(s[index])

    song = to_json({},sid,'read')[index]
    clear()
    print('Saving',a+' by '+b+'\n')
    return song['lyrics']
# def genius_to_lyrics(index,split_lyrics):
  
#   song = to_json({},sid,'read')
#   list_of_song = song.keys()

#   if index in list_of_song:
#     url  = song[index]['url']
#     print(song[index]['song_name']+' already saved\n')
#     return song[index]['lyrics']
#   else:
#     try:
#       a,b = done_dl[index]['file_name'].replace('by\u200b','by').split(' by ')
#       # b = re.sub(",","",b)
#     except ValueError:
#       a=done_dl[index]['file_name']
#       b=''
#     search_url = "https://genius.com/search?q="+quote(a+' '+b.split(',')[0])
#     print('CLICKKKKKK',search_url)
#     print('\n\n')
#     if desparate_time in empty_list:
#       url = input('Enter link for '+done_dl[index]['file_name']+'\n')
#     else:
#       url = ''
#     enter_song_data_from_genius_link(url,index,split_lyrics)

#     song = to_json({},sid,'read')[index]
#     clear()
#     print('Saving',a+' by '+b+'\n')
#     return song['lyrics']
done_dl = to_json({},dmd,'read')

In [ ]:
# st = 'Killer (Remix) Lyrics'
# kl = 'Killer (Remix) LyricsKOREAN\nI am the killer\nLyrics Lyrics lyrics'
# kl = re.sub(st+r"(\w*)","",kl)
# # kl.replace(st,'')
# kl

In [ ]:
# st = st.replace("(","\(")
# st = st.replace(")","\)")

# kl = re.sub(st+r"(\w*)","",kl)
# kl

In [ ]:
def enter_song_data_from_genius_link(genius_link,index,split_lyrics):

  # if desparate_time == 1:
  #   deezer_song_data_for_desparate_times()
  #   return '\nCollecting data from Deezer cause genius.com is being a bitch...\n\n'


  # genius_link = 'https://genius.com/Grimes-4m-lyrics'
  # index = index

  song = to_json({},sid,'read')
  # song = {}
  search_term = ''
  search_term = genius_search(genius_link)

  song_id = 0
  while song_id == 0:
    h = genius.search_songs(search_term)

    for item in h['hits']:
      song_id = 0
      if item['result']['url']==genius_link:
        song_id = item['result']['id']
        break
    
    if song_id == 0:
      search_term = input('Did not find shit for '+search_term+'\n\nEnter custom search: ')

  d = genius.song(song_id)
  feat = prod = date = ''
  song_name = zero_width_cleaner(d['song']['title'],'')
  date = d['song']['release_date']
  song_art = d['song']['song_art_image_url']
  main_artist = zero_width_cleaner(d['song']['primary_artist']['name'],'')
  main_artist = re.sub("&","&\u200b",main_artist)
  featuring = d['song']['featured_artists']
  producers = d['song']['producer_artists']

  
  lyrics = zero_width_cleaner(genius.lyrics(song_id),1).strip()
  st = song_name
  kl = lyrics
  st = st.replace("(","\(")
  st = st.replace(")","\)")

  lyrics = re.sub(st+r" Lyrics(\w*)","",kl)
  # lyrics = .replace(song_name+' Lyrics','')
  if split_lyrics == 1:
    
    lyrics = bird_cage(lyrics,'')
    
  

  if featuring != []:
    feat = 'Featuring\u200b'+get_elements(featuring)

  if producers != []:
    prod = 'Produced\u200b by\u200b'+get_elements(producers)

  if d['song']['album'] != None:
    album = d['song']['album']['name']
    album_art = d['song']['album']['cover_art_url']
    for tracks in genius.album_tracks(d['song']['album']['id'])['tracks']:
      if song_id == tracks['song']['id']:
        track_number = tracks['number']
    
    if track_number == None:
      album = 'On\u200b'+' '+album
    if track_number is not None:
      album = '#'+str(track_number)+' on\u200b '+album
    
  if d['song']['album'] is None or d['song']['album']['name'][-1] in ['*']:
    album_art = song_art
    album = 'Single'

  if date is not None:
    date = datetimedate.fromisoformat(date)

    date = "Released\u200b on\u200b "+date.strftime("%B "+num_to_ordinal(date.day)+", %Y")

  
  if index in empty_list:
    return lyrics
    
  song[index]={}
  song[index]={
      'song_name':song_name,
      'feat':feat,
      'prod':prod,
      'date':date,
      'album':album,
      'cover_art':album_art,
      'song_art':song_art,
      'main_artist':main_artist,
      'lyrics':lyrics,
      'song_id':song_id,
      'url':genius_link,
      'mp3_file_index':index
  }
  # to_json(song,sid,'write')


  to_json(song,sid,'write')

  return song[index]



In [ ]:

def font_file_list():
  # frequent = ['NotoSansMonoCJ','NotoNaksh','NotoSerifBengali','NotoSans-Regular']
  regular_files = []
  # for files in os.listdir(font_path):
  #   for item in frequent:
  #     if files.startswith(item):
  #       regular_files.append(files)

  for files in os.listdir(font_path):
    if files.startswith('Noto'):
      regular_files.append(files)
  return regular_files 

def check_font(font,character):
  font = ttfont(font)
  for table in font['cmap'].tables:
    if ord(character) in table.cmap.keys():
      return True
  return False

import manimpango
try:
  if api_caller_font ==1:
    print('FONTS ARE ALREADY REGISTERED. ^-^')
except NameError:
  
  print('INSTALLING FONTSSSSS.....')
  # import manimpango
  manimpango.register_font(m)
  for item in font_file_list():
    manimpango.register_font(font_path+item)
  api_caller_font = 1


In [ ]:

def printer(word, font,color, word_by_word,color_dictionary,is_this_lyrics):
  
  if color_dictionary in empty_list:
    if word_by_word ==1:
      color_dictionary = colors(word)

  if color in empty_list:
    color = (154,154,154)

  if font in empty_list:
    font = default
  
  
  if word in allowed_punct:
    font = default
  
  if font in ['NotoSans-Regular.ttf']:
    font_type = ImageFont.truetype(m,size=109,layout_engine=ImageFont.LAYOUT_RAQM)
    # if word[-1] in empty_list:
    # word = word.strip()+' '
  else:
    # if word[-1] in empty_list:
    
    font_type = ImageFont.truetype(n+font,size=109,layout_engine=ImageFont.LAYOUT_RAQM)
  
  # if font in sa[1]:
  #   font_type = ImageFont.truetype(n+'Kalpurush.ttf',size=116,layout_engine=ImageFont.LAYOUT_RAQM)

  if word_by_word ==1:
    m1=o1=0
    word = word.strip()
    c= len(word.split())
    c1 = 0
    for w in word.split():
      
      if w not in empty_list:
        c1+=1
        if c1<c:
          a1,b1= font_type.getsize(w+' ')
        else:
          a1,b1= font_type.getsize(w)
        m1+=a1
        if b1>o1:
          o1=b1
    size = (m1,o1)

  else:
    size = font_type.getsize(word)

  if word in allowed_punct:
    size = font_type.getsize(word)
  # size = font_type.getsize(word+' ')
  
  image = Image.new('RGBA',size,(0,0,0,0))
  draw = ImageDraw.Draw(image)
  margin = offset = 0
  
  if word_by_word == 1:
    if font in ['NotoNaskhArabic-Regular.ttf']:
      lists = word.split()[::-1]
        
    else:
      lists = word.split()
    for words in lists:
      
      if strips_words(words.strip().lower(),0,is_this_lyrics) not in color_dictionary.keys():
        # print(words, 'no color')
        draw.text((margin,offset),words,font=font_type,fill=color,embedded_color=True)  
      else:
        # print(words,color_dictionary[strips_words(words.strip().lower(),0)] )
        draw.text((margin,offset),words,font=font_type,fill=color_dictionary[strips_words(words.strip().lower(),0,is_this_lyrics)],embedded_color=True)
      margin += font_type.getsize(words+' ')[0]
  else:
    draw.text((margin,offset),word,font=font_type,fill=color,embedded_color=True)

  return image


In [ ]:
def main_printer(line,font,color,word_by_word,non_proceeding_space,color_dictionary,background,is_this_lyrics):
  
  # if is_this_lyrics==1:
  line = zero_width_cleaner(line,is_this_lyrics)
  default = 'NotoSans-Regular.ttf'
  # print(line)
  # display(line)
  w = ""
  blank_space = printer(' ','','','','','')
  if font in empty_list:
    fon1 = default
  else:
    fon1 = font
  # fon1 = default
  if color in empty_list:
    # print('No COLOR GIVEN')
    color = (154,154,154)

  img_array= []
  if non_proceeding_space==1:
    img_array.append(blank_space)

  # list_of_words = fontaine(line)
  # print(list_of_words,'before')
  if background in empty_list:
    list_of_words = fontaine(line)
    for i in range(len(list_of_words)):
      word,font = list_of_words[i]['word'],list_of_words[i]['font']
      # display(word)
      im = printer(word,font,color,1,color_dictionary,is_this_lyrics)
      # display(im)
      img_array.append(im)
      
      if i <len(list_of_words):
        try:
          # print(list_of_words[i]['word'][-1])
          if list_of_words[i+1]['word'][0] not in non_space_punct and list_of_words[i+1]['word'][0] not in right_side:
            if list_of_words[i]['word'][-1] not in left_side:      
              img_array.append(blank_space)
        except IndexError:
          break
  
  else:
    im = printer(line,'','','','','')
    img_array.append(im)

  
  if non_proceeding_space==1:
    img_array.append(blank_space)

  margin_length = 0
  offset_length = []
  for image in img_array:
    margin,offset = image.size
    margin_length +=margin
    offset_length.append(offset)
  # print(margin_length,max(offset_length))
  im = Image.new('RGBA',[margin_length, max(offset_length)],(0,0,0,0))

  margin=offset = 0
  for image in img_array:
    
    im.alpha_composite(image,(margin,offset))
    margin += image.size[0]


  # display(line)
  return im, font,im.size

# song_card(res,lyr)

In [ ]:

def iresize_height(res,used_image,scale):
  height = res
  w,h = used_image.size
  w = (w/h)*height*scale
  h = height*scale

  return int(w), int(h)

def iresize(res,used_image,scale):
  width = res
  w,h = used_image.size
  h = (h/w)*width*scale
  w = width*scale
  if w<1:
    w=1
  return int(w), int(h)

In [ ]:

def deezer_lyrics_splitter(lyrics_sync,split_lyrics):
  print('Splitting deezer lyrics')
  a = []
  b = ""
  for item in lyrics_sync:
    if item['line'] not in empty_list:
      # print(item['line'])
      if split_lyrics ==1:
        item['line'] = w = bird_cage(item['line'],'').strip()
      else:
        w = item['line']
      a.append(item)
      b+=w+'\n'
  print('Splitting Done')
  return b,a


In [ ]:
def makes_lyrics(res,lyrics,d):
  a = re.sub('\n\n\n','\u200b',lyrics)
  a = re.sub('\n\n','\u200b',a)
  a = re.sub('\n',' ',a).strip()
  a = re.sub('\u200b','\n',a)

  gt= a

  s = re.sub(r"[\u2026]"," ", gt) # strips the ellipses
  
  gt = s
  captions= s

  stripped_lines = []
  count = total_character = 0
  for lines in captions.split('\n'):
    if len(lines)!=0 and lines !=' ' and lines != '':
      count+=1
      total_character += len(lines.strip())
  
  brrt = int(total_character/31)
  if brrt>80:
    brrt = int(total_character/40)
  if brrt<10:
    brrt = int(total_character/20)
  
  
  
  list_of_lines = limiting_line_space(captions,brrt)
  
  img_array = []
  fon1 = default
  color = (154,154,154)


  margin = []
  offset = 0
  count = 0
  for sentence in list_of_lines:
    count+=1
    img,fon1,size = main_printer(sentence,fon1,color,1,1,color_dictionary,"",1)
    print('('+str(count)+'/'+str(len(list_of_lines))+')',sep,sentence,sep,fon1)
    # img = img.resize(iresize_height(height,img,1))
    if d == 1:
      display(img)
    if count%3 ==0:
      clear()
    marg,off= img.size
    margin.append(marg)
    offset+=off

    img_array.append(img)

  
  dim = max(margin),offset

  main = Image.new('RGBA',dim,(0,0,0,0))
  main = main.convert('RGBA')

  margin = offset = 0


  for im in img_array:
    margin = 0
    margin = int((main.size[0]-im.size[0])/2)
    main.alpha_composite(im,(margin,offset))

    offset+=im.size[1]

  uno = Image.new('RGBA',[res,res],(0,0,0,0))
  uno = uno.convert('RGBA')


  if main.size[0]>res:
    main = main.resize(iresize(res,main,1))
    

  if main.size[1]>res:
    main = main.resize(iresize_height(res,main,1))
    
  clear()

  if main.size[0]<res:
    uno.alpha_composite(main,(int((res-main.size[0])/2),0))
  else:
    uno.alpha_composite(main,(0,int((res-main.size[1])/2)))
  return  uno


In [ ]:

def existing_lyrics_splitter():
  a = to_json({},slsd,'read')
  w=''
  for item in a[index].split('\n'):
    # print(item)
    c,d = item.split(']')
    # display(d)
    if d not in empty_list and d not in ["\u200b"]:
      d= bird_cage(zero_width_cleaner(d,1),'').split('\n')[1].strip()
      # print(d)
      display(c+']'+d+'\n')
    else:
      d = "\u200b"
    w+=c+']'+d+'\n'
    
  clear()
  # print('\n')
  display(w)
  # return w



In [ ]:
# rogue_killer()



In [ ]:
def strips_words(string,split,is_this_lyrics):
  # string = norm(string).decode('utf-8')
  if string in lst:
    return string
  a = zero_width_cleaner(string,is_this_lyrics)
  
  a = re.sub('\n',' ',a).strip()
  a = re.sub("  "," ",a)
  a = a.lower()
  
  a = re.sub(r"(?<!\w)\'|\'(?!\w)",'',a) # deletes trailing and preceeding apostrophe
  a = re.sub(r"-(?<!\w)\-|\-(?!\w)-",'',a) 
  a = re.sub(r"(?<!\w)\’|\’(?!\w)",'',a) # deletes trailing and preceeding apostrophe
  
  a = re.sub(r"(?<!\w)\“|\“(?!\w)",'',a)
  a = re.sub(r"(?<!\w)\”|\”(?!\w)",'',a)
  a = re.sub(r"(?<!\w)\‘|\‘(?!\w)",'',a) # deletes trailing and preceeding apostrophe
  # a = re.sub(r"[^\S]?(\(.*?\))[^\S]?", r" \1 ", a).strip() ## puts space before parenthesis
  if is_this_lyrics not in empty_list:
    a = a.translate(str.maketrans("","",p1))

  
  # a= re.sub(r'-(?!\w)|(?<!\w)-','',a)
  


  if split==1:
    a = a.split()

  return a

# song_card(res,lyr)


In [ ]:
def download_song_arts():
  os.chdir(placeholder_images)
  extension_of_image = '.jpg'
  song = to_json({},sid,'read')[index]
  if song['cover_art'] in empty_list or song['song_art'] in empty_list or len(song['song_art'].split('/'))==5 or len(song['cover_art'].split('/'))==5:
    if 'deezer' in get_sd()[index]['link']:
      track = deezer.get_track(int(get_sd()[index]['link']['deezer'].split('/')[-1]))
      data = track['tags']['_albumart']['image']

      with open(song_name+'_song_art'+extension_of_image, 'wb') as f:
        f.write(data)
      
      with open(song_name+'_cover'+extension_of_image, 'wb') as f:
        f.write(data)

      cover = Image.open(song_name+'_song_art'+extension_of_image)
      os.chdir('/content')
      return cover, cover
    
    else:
      spotify_song_data_for_desparate_times()
  
      song = to_json({},sid,'read')[index]
  
  cover = requests.get(song['cover_art'],allow_redirects=True)
  ext = os.path.splitext(song['cover_art'])[-1]
  c = open(song_name+'_cover'+ext,'wb').write(cover.content)
  if ext not in ['.jpg']:
    im = Image.open(song_name+'_cover'+ext,mode='r').convert('RGB')
    im.save(song_name+'_cover'+extension_of_image)
    im=''
    os.remove(song_name+'_cover'+ext)
  song_art = requests.get(song['song_art'],allow_redirects=True)
  ext = os.path.splitext(song['song_art'])[-1]
  d = open(song_name+'_song_art'+ext,'wb').write(song_art.content)
  if ext not in ['.jpg']:
    im = Image.open(song_name+'_song_art'+ext,mode='r').convert('RGB')
    im.save(song_name+'_song_art'+extension_of_image)
    os.remove(song_name+'_song_art'+ext)
    im=''

  cover = Image.open(song_name+'_cover'+extension_of_image)
  song_art = Image.open(song_name+'_song_art'+extension_of_image)
  os.chdir('/content')
  return cover,song_art



In [ ]:
def limiting_line_space(data,width):
  lines = ''

  for line in data.split('\n'):
    if len(line) > width:
      w = textwrap.TextWrapper(width=width,break_long_words=False,drop_whitespace=False,break_on_hyphens=False)
      line = '\n'.join(w.wrap(line))
    lines += line+'\n'
  lines  = lines.split('\n')

  lines2= []

  for line in lines:
    if line.strip() not in empty_list:
      lines2.append(line.strip())

  lines = lines2
  line_length = []

  for line in lines:
    line_length.append(len(line))

  lines_filled  =[]

  for i in range(len(lines)):
    lines_filled.append('{message:{fill}{align}{width}}'.format(message=lines[i],
                                                                fill = ' ',
                                                                align='<',
                                                                width=width))
  return lines_filled





In [ ]:
def combined_data(res):
  list_of_keys = ['song_name','main_artist','feat','prod','album','date']

  l = []
  main_res =res
  for keys in list_of_keys:
    if song[keys] not in empty_list and song[keys] != None:
      l.append(keys)

  img_array = []
  margin = []
  # song['song_name']=(re.sub("\(",'\n(',song['song_name']))
  offset = 0
  resize_dictionary = {'song_name':{'multiplier':4,'color':(255,255,100),'width':35},
                      'main_artist':{'multiplier':3.5,'color':(255,255,255),'width':50},
                      'feat':{'multiplier':3,'color':(125,143,232),'width':50},
                      'prod':{'multiplier':2.7,'color':(125,143,232),'width':50},
                      'album':{'multiplier':2.4,'color':(153,167,238),'width':50},
                      'date':{'multiplier':2.3,'color':(237,237,237),'width':50}
  }

  # print(len(l))
  if len(l)>5:

    resize_dictionary = {'song_name':{'multiplier':4,'color':(255,255,100),'width':45},
                        'main_artist':{'multiplier':3.5,'color':(255,255,255),'width':50},
                        'feat':{'multiplier':3,'color':(125,143,232),'width':60},
                        'prod':{'multiplier':2.7,'color':(125,143,232),'width':65},
                        'album':{'multiplier':2.4,'color':(153,167,238),'width':50},
                        'date':{'multiplier':2.3,'color':(237,237,237),'width':50}
    }
  # else:
  #   resize_dictionary = {'song_name':{'multiplier':4,'color':(255,255,100),'width':25+10},
  #                       'main_artist':{'multiplier':3.5,'color':(255,255,255),'width':35},
  #                       'feat':{'multiplier':3,'color':(125,143,232),'width':40+10},
  #                       'prod':{'multiplier':2.7,'color':(125,143,232),'width':40+10},
  #                       'album':{'multiplier':2.4,'color':(153,167,238),'width':40+10},
  #                       'date':{'multiplier':2.3,'color':(237,237,237),'width':50}}

  # if len(song['main_artist'])<10:
  #   resize_dictionary['song_name']={'multiplier':4,'color':(255,255,100),'width':25}
  
  # if len(song['song_name'])<10 and len(song['main_artist']) > 25:
  #   resize_dictionary['main_artist']={'multiplier':3.5,'color':(255,255,255),'width':25}
    
  # if len(song['song_name'])<10 and len(song['main_artist'])<10:
  #   resize_dictionary['prod']={'multiplier':2.7,'color':(125,143,232),'width':40}
  if song['album']=='Single':
    resize_dictionary['album']={'multiplier':3,'color':(153,167,238),'width':6}
  
  # if len(l)==6:
  #   resize_dictionary['song_name']={'multiplier':4,'color':(255,255,100),'width':40}
  # if len(song['main_artist'])>50 and len(song['song_name'])>10:
  #   resize_dictionary['main_artist']={'multiplier':3.5,'color':(255,255,255),'width':40}
  # display(resize_dictionary)
  img_array  = []
  img_array1=[]
  fon1 = default
  color= (154,154,154)
  # res = res*(512/1080)
    # res = 512
  res = int(res*(1-(380/1080)))
  # key = 'song_name'
  for key in l:
    data = extractor(song[key])
    if data in empty_list:
      continue

    # if key == 'song_name':
    #   data=(re.sub("\(",'\n(',song['song_name']))


    # data = zero_width_cleaner(data)
    width = resize_dictionary[key]['width']
    col = resize_dictionary[key]['color']
    mul = resize_dictionary[key]['multiplier']
    
    # print(res)
    d = limiting_line_space(data,width)

    # img = main_
    lst = ['&\u200b','Featuring\u200b','Released\u200b', 'on\u200b','Produced\u200b', 'by\u200b','On\u200b']

    color_dictionary = {}
    
    for words in data.split():
      if words in lst:
        color_dictionary[strips_words(words,0,'')] = color
      else:
        color_dictionary[strips_words(words,0,'')] = col
    count = 0

    # display(color_dictionary)
    # if len(d)>1:
    #   mul = mul*.8
    save_size = ''
    img_array=[]
    for line in d:
      # display(line,color_dictionary)
      count+=1
      # if count==1:
        # blank = printer(line,'',(255,255,255,0),'','','')
        # blank = blank.resize(iresize_height(res,blank,mul))
        # save_size = blank
      # display(line)
      
      # blank = printer(line,'','',(255,255,255,0),'')

      # if count>1:
      #   img,fon1,size = main_printer(line,fon1,color,1,'',color_dictionary,'')  

      # blank = blank.resize(iresize_height(res,blank,mul))
      img,fon1,size = main_printer(line,fon1,color,1,'',color_dictionary,'','')
      # img = img.resize(iresize_height(512,img,mul),reducing_gap=4)
  
      # display(img)
      img_array.append(img)
    
    margin = [] 
    offset = 0
    for img in img_array:
      margin.append(img.size[0])
      offset+=int(img.size[1]*1)
      

    image = Image.new('RGBA',[max(margin),offset],(0,0,0,0))

    margin = offset = 0
    for img in img_array:
      # if img.size[0]>res:
      #   img = img.resize(iresize(res,img,1),reducing_gap=4)
      image.alpha_composite(img,(margin,offset))
      # print(img.size)
      
      offset+=int(img.size[1]*1)
      # if image.size[0]>res:
      #   image = image.resize(iresize(res,image,mul),reducing_gap=4)
      # if image.size[1]>res*0.1*count:
      #   image = image.resize(iresize_height(res*0.1*count,image,mul),reducing_gap=4)
    image = image.resize(iresize_height(res*0.1*count,image,mul),reducing_gap=4)
    # print(res,sep,res*0.1*count,sep,mul,sep,image.size)
    # display(image)
    img_array1.append(image)
    
  
    


      
      
  # clear()
  # print('---------------------')
  margin = [] 
  offset = 0
  for img in img_array1:
    margin.append(img.size[0])
    offset+=int(img.size[1]*1.1)
    

  image = Image.new('RGBA',[max(margin),offset],(0,0,0,0))

  margin = offset = 0
  for img in img_array1:
    # if img.size[0]>res:
    #   img = img.resize(iresize(res,img,1),reducing_gap=4)
    image.alpha_composite(img,(margin,offset))
    # print(img.size)
    offset+=int(img.size[1]*1.1)
  if image.size[0]>main_res:
    image = image.resize(iresize(main_res,image,1),reducing_gap=4)
  return image

# combined_data(res)
# card = song_card(res,lyr)
# test('')
# makes_video()

In [ ]:
def extractor(line):
  w= ''
  for item in fontained(line):
    w+=item['word']+' '
  return  w.strip()

In [ ]:
from datetime import datetime
def deezer_lyrics_to_lrc():
  lrc = to_json({},slsd,'read')
  # lrc = normalize_keys(lrc)
  lrc = list(lrc.keys())
  
  if lyrics_sync == '':
  
    if index in lrc:
      return name_of_the_song+' lrc already saved'
  
    
    s = genius_to_lyrics(index,0,'')
    # s = re.sub(r"[\[].*?[]]"," ",grr).strip()
    word = ''
    for i in s.split('\n'):
      if i not in empty_list:
        line = i
        # t = i.split(']')[0]+']'
        trans,pron=translating(line)
        
        if (trans,pron)==(False,False):
          trans,pron = timer_for_google_translate(line)

        # while (trans, pron) == (False,False):
        #   trans,pron=translating(line)
        
        

        if line not in ["\u200b"] and norm(pron)!=norm(line):
          word+=line+separator+pron+'\n\u200b\n'
        else:
          word+=line+'\n\u200b\n'
      

    word = word.strip()
    # print('paste below to and sync it','https://lrc-maker.github.io/#/','\n\n')
    # display(word)
    # print('\n\n')
    # print(done_dl[index])
    # print('\n')
    edit_lrc(1)
    lyr_data = convert_lrc_to_lyr_data()

  else:
    if index in lrc:
      return name_of_the_song+' lrc already saved'
  


    # return False
    # if change == 1:

  
    for i in lyrics_sync:
      if i['line']=='':
        lyrics_sync.remove(i)



    # lyr_data = 
    # for i in lyrics_sync:
    # i=lyrics_sync[10]
    lyr_data = {}
    f=0
    while len(list(lyr_data.keys())) < len(lyrics_sync)*2:
      
      lyr_data = {}
      for i in lyrics_sync:
        timestamp = int(i['milliseconds'])
        line = i['line']
        
        blank = timestamp+int(i['duration'])-f
        
        timestamp = int((timestamp/1000)*30)
        blank = int((blank/1000)*30)
        lyr_data[timestamp]=line
        lyr_data[blank]="\u200b"
      
      f+=1

  b= lyr_data
  brr = ''
  for i in b:
    s = datetime.fromtimestamp(i/30).strftime("%M:%S")
    ms = datetime.fromtimestamp(i/30).strftime("%f")
    ms = '.'+ms
    ms = float(ms)
    ms = round(ms,3)
    ms = str(ms).split('0.')[-1]
    time = s+'.'+ms
    line = b[i]

    brr+='['+time+']'+line+'\n'

  brr = brr.strip('\n')
  print('paste below in here','https://lrc-maker.github.io/#/')
  display(brr)
  h=to_json({},'song_lyrics_sync_dictionary','read')
  h[index]=brr
  h=to_json(h,'song_lyrics_sync_dictionary','write')
  clear()
  return name_of_the_song+' lyrics sync data saved'

def convert_lrc_to_lyr_data():
  lrc = to_json({},slsd,'read')
  # lrc = normalize_keys(lrc)
  lrc_data = lrc[index]
  
  # if index in lrc:
  #   lrc_data = to_json({},'song_lyrics_sync_dictionary','read')[index]

  
  
  # if lrc_data == '':

    # lrc_data = normalize_keys(to_json({},'song_lyrics_sync_dictionary','read'))[norm(name_of_the_song)]
  

    # lrc_data
  i =0
  lyr_data = {}
  # import math
  lst = lrc_data.split('\n')
  f=0
  lyr_data = {}
  f = 0 
  frames= 30

  lyr_data = {}
  for i in range(len(lst)):
    time,line = lst[i].strip().split(']')
    time = time.split('[')[-1]
    # print(time,line)
    m,s = time.strip('[').split(':')
    m,s = int(m),float(s)
    timestamp = int((m*60+s)*frames)
    if line in ['\u200b']:
      continue

    lyr_data[timestamp] = line.strip()

  times = list(lyr_data.keys())

  new_lyr = {}
  for i in range(len(lst)):
    
    time,line = lst[i].strip().split(']')
    if line in ['\u200b']:
      time = time.split('[')[-1]
      # print(time,line)
      m,s = time.strip('[').split(':')
      m,s = int(m),float(s)
      timestamp = int((m*60+s)*frames)
      # print(timestamp)
      while timestamp in times or timestamp-1 in times or timestamp-2 in times:
        # print(timestamp, 'was in times')
        timestamp -=3
    else:
      try:
        m,s = time.strip('[').split(':')
        m,s = int(m),float(s)
        timestamp = int((m*60+s)*frames)
        new_lyr[timestamp]=line
      except IndexError:
        continue
      continue

    new_lyr[timestamp] = line.strip()
  lyr_data = new_lyr

  return lyr_data


def edit_lrc(genius_lyrics):
  if lyrics_sync != '':
    deezer_lyrics_to_lrc()
    s = to_json({},slsd,'read')
    s=s[index]
  
  # s = to_json({},slsd,'read')[done_dl[index]['file_name']]


  
  s = to_json({},slsd,'read')
  try:
    s=s[index]
  except KeyError:
    s[index]=''
  
  if genius_lyrics==1:
    s = genius_to_lyrics(index,0,'')
    s = re.sub(r"[\[].*?[]]"," ",s).strip()
    s = re.sub(r"[\[].*?[]]"," ",s).strip()
    word = ''
    for i in s.split('\n'):
      if i not in ["",'',' ']:
        line = i.strip('\n')
        # t = i.split(']')[0]+']'
        # t=''
        pron=translating(line)[1]

        if norm(pron.replace("'","").replace("’",""))!=norm(line.replace("'","").replace("’","")):
          word+=line+separator+pron+'\n'+"\u200b"+'\n'
        else:
          word+=line+'\n'+"\u200b"+'\n'
  
  if genius_lyrics!=1:
    word = ''
    for i in s.split('\n'):
      if i not in empty_list:
        line = i.split(']')[-1]
        t = i.split(']')[0]+']'
        pron=translating(line)[1]
        
        if line not in ["\u200b"] and norm(pron.replace("'","").replace("’",""))!=norm(line.replace("'","").replace("’","")):
          word+=t+line+separator+pron+'\n'
        else:
          word+=t+line+'\n'
    
        
  clear()
  print('Editing sync for '+done_dl[index]['file_name'])
  sound = AudioSegment.from_file(done_dl[index]['location'])
  sound.export("/content/SYNC_MUSIC/FOR_SYNC__"+done_dl[index]['file_name']+".mp3", format="mp3", bitrate="128k")
  shutil.copy(done_dl[index]['location'],'/content/SYNC_MUSIC/'+done_dl[index]['file_name']+'.mp3')
  print('paste below then edit and sync it','https://lrc-maker.github.io/#/','\n')
  print('The mp3 file is in the SYNC_MUSIC folder for quick download\n\n')
  
  display(word)
  print('\n\n')
  print(done_dl[index])
  print('\n')
  brr = input('\n\nEnter lrc_data [ONLY THE LYRICS] \n'+done_dl[index]['file_name']+' '+'\n')
  os.remove('/content/SYNC_MUSIC/FOR_SYNC__'+done_dl[index]['file_name']+'.mp3')
  lrc_data = brr
  i =0
  lyr_data = {}
  # import math
  lst = lrc_data.split('[')[1:]
  f=0
  while len(list(lyr_data.keys())) <  len(lst):
    lyr_data={}
    for i in range(len(lst)):
      time,line = lst[i].split(']')
      line = line.split(separator)[0].strip()
      m,s = time.strip('[').split(':')
      m,s = float(m),float(s)
      timestamp = int((m*60+s)*30)
      if line in ["\u200b","",'']:
        # print('line is blacnk',f)
        timestamp = int((m*60+s)*30)-f

      lyr_data[timestamp]=line.strip()
    # print('lyr_data lenght',len(list(lyr_data.keys())),'f',f,'lst length',len(lst))
    # clear()
    f+=1

  # lyr_data
  b= lyr_data
  brr = ''
  for i in b:
    s = datetime.fromtimestamp(i/30).strftime("%M:%S")
    ms = datetime.fromtimestamp(i/30).strftime("%f")
    ms = '.'+ms
    ms = float(ms)
    ms = round(ms,3)
    ms = str(ms).split('0.')[-1]
    time = s+'.'+ms
    line = b[i]

    brr+='['+time+']'+line+'\n'

  brr = brr.strip('\n')
  h=to_json({},'song_lyrics_sync_dictionary','read')
  h[index]=brr
  h=to_json(h,'song_lyrics_sync_dictionary','write')
  clear()
  return done_dl[index]['file_name']+' lyrics sync data saved'

In [ ]:
def song_card(res,lyr):  
  cover,song_art = download_song_arts()

  # res = 512
  song_art_resize = int(res*(0.2))
  song_art_location = int(res*0.028)
  fill = int((10/270)*song_art_resize)
  lyr = lyr.resize([res,res],reducing_gap=4)
  # data = combined_data(res)
  color= (154,154,154)
  card = black = Image.new('RGBA',[res,res],(0,0,0)).convert('RGBA')
  # h = making_pattern(matrix(lyrics,1,color_dictionary),'x',res)
  cover = cover.resize([res,res],reducing_gap=4).convert('RGBA')
  song_art = song_art.resize([song_art_resize,song_art_resize],reducing_gap=4).convert('RGBA')
  # song_art.alpha_composite(h.resize([song_art_resize,song_art_resize]),(0,0))
  song_art = ImageOps.expand(song_art,border=(fill,fill,fill,fill),fill=color)
  card = Image.blend(card,cover,alpha=0.4)
  # card.alpha_composite(h,(0,0))
  card.alpha_composite(lyr.convert('RGBA'),(0,0))
  
  # card.alpha_composite(h,(0,0))

  card = Image.blend(card,black,alpha=0.75)

  card.alpha_composite(song_art,(song_art_location,song_art_location))

  
  
  # data_loc = song_art.size[0]+song_art_location+fill+int(song_art_location/2),int(song_art_location-fill*6)
  data_resize = int(res-(song_art_resize+song_art_location*2))
  data_loc = int(song_art_resize+song_art_location*2)                      
  # print(data_resize)
  data = combined_data(data_resize)
  # print('original DATA SIZE:',data.size)
  jk = data.size[1]
  data_resize_h = int(song_art_resize+song_art_location*2.5)
  print('height: ',data_resize_h)
  
  
  # data = data.resize(iresize(data_resize,data,1),reducing_gap=4)
  data = data.resize(iresize_height(data_resize_h,data,1),reducing_gap=4)
  # # print('height resize',data.size)
  
  
  if data.size[0]>res-data_loc:
    
    data = data.resize(iresize(data_resize,data,.97),reducing_gap=4)
    # print('width resize',data.size)
  clear()
  d = res-data_loc,data_resize_h
  # data = data.resize(iresize_height(data_resize_h,data,1),reducing_gap=4)
  offset_h = ((fill+data_resize_h-data.size[1])/2)-fill
  print('DATA SIZE: ',data.size,'\nThreshold:',d,'\nOffset_h:',math.ceil(offset_h),'fill:',fill)
  card.alpha_composite(data,(data_loc,math.ceil(offset_h)))
  # print('jk',offset_h)
  # print(res,sep,data_resize,data_resize_h)

  print('song_art','size',song_art_resize,'location',song_art_location)
  print('song_data','size',data.size,'location',data_loc,math.ceil(offset_h))
  return card

# card = song_card(res,lyr)
# test('')
# # makes_video()

In [ ]:
def making_pattern(image,pattern,res):
  if pattern == 'none':
    return image
  h = int(image.size[0]*2)
  new = Image.new('RGBA',[h,h],(0,0,0,0))

  if pattern == 'box':
    c = 0
    d = 270
    a = 90
    b = 180

  if pattern == 'x':
      
    a = 0
    b = 270
    c = 90
    d = 180
    # a,b,c,d = 180,90,270,0

  new.paste(image.rotate(a),(0,0))
  new.paste(image.rotate(b),(int(h/2),0))
  new.paste(image.rotate(c),(0,int(h/2)))
  new.paste(image.rotate(d),(int(h/2),int(h/2)))
  new = new.resize([res,res],reducing_gap=3)
  return new



In [ ]:
def elements_for_the_mv(res):
  im_file = placeholder_images+song_name
  extension_of_image = '.jpg'
  cover = Image.open(im_file+'_cover'+extension_of_image)
  song_art = Image.open(im_file+'_song_art'+extension_of_image)
  song_art = song_art.resize([res,res],reducing_gap=4)
  cover = cover.resize([res,res],reducing_gap=4)
  
  a = re.sub('\n\n\n','\u200b',lyrics)
  a = re.sub('\n\n','\u200b',a)
  a = re.sub('\n',' ',a).strip()
  a = re.sub('\u200b','\n',a)

  gt= a

  s = re.sub(r"[\u2026]"," ", gt) # strips the ellipses

  gt = s
  captions= s

  stripped_lines = []
  count = total_character = 0
  for lines in captions.split('\n'):
    if len(lines)!=0 and lines !=' ' and lines != '':
      count+=1
      total_character += len(lines.strip())

  brrt = int(total_character/31)
  if brrt>80:
    brrt = int(total_character/40)
  if brrt<10:
    brrt = int(total_character/20)



  list_of_lines = limiting_line_space(captions,brrt)

  img_array = []
  fon1 = default
  color = (154,154,154)


  margin = []
  offset = 0
  count = 0
  for sentence in list_of_lines:
    count+=1
    
  # from manim import *
  black = Image.new('RGBA',[1000,1000],(0,0,0,0))
  black = ImageMobject(black).scale_to_fit_width(config.frame_width)
  c = ImageMobject(cover).scale_to_fit_width(config.frame_width)
  t = BackgroundRectangle(black,color=hexcolor(0,0,0),fill_opacity=.75)
  s_art = ImageMobject(song_art)
  s_art.scale_to_fit_width(config.frame_width)
  s_art.scale(0.205)
  # s_art.to_edge(UL,buff=.28)
  s_art.align_on_border(UP,buff=0.45*(2.915555555/s_art.height))
  s_art.align_on_border(LEFT,buff=0.40)
  
  # k = BackgroundRectangle(s_art,color=hexcolor(154,154,154),buff=0.1,stroke_width=1)
  k=SurroundingRectangle(s_art,color=hexcolor(154,154,154),buff=0.01,stroke_width=10)
  s_art.move_to(k.get_center()+[.015,-.0125,0]).scale_to_fit_width(k.width*.97)
  t1= BackgroundRectangle(t,color=hexcolor(0,0,0),stroke_opacity=0,fill_opacity=.73)

      
  g0 = VGroup()
  # color_dictionary = colors(lyrics)
  for line in list_of_lines:
    a = MarkupText(markup_texter(line,color_dictionary,1).replace('&','&amp;'),font_size=15)

    # clear()
    g0+=a
  g0.arrange(DOWN,buff=0.07)
  # k0=VGroup()
  # for item in g0:
  #   if k0.width > g0.width:
  #     k0+=item.scale_to_fit_width(g0.width)
  #   else:
  #     k0+=item
  # g0 = k0.arrange(DOWN)
  box = Rectangle(  # create a box
          height=config.frame_height, width=config.frame_width, fill_color=hexcolor(255,255,100), 
          fill_opacity=0.5, stroke_color=hexcolor(255,255,100)
      )
  g0.scale_to_fit_height(config.frame_height*0.98)
  if g0.width > config.frame_width:
    g0.scale_to_fit_width(config.frame_width*.98)
  list_of_keys = ['song_name','main_artist','feat','prod','album','date']

  l = []
  res=600
  main_res =res
  for keys in list_of_keys:
    if song[keys] not in empty_list and song[keys] != None:
      l.append(keys)

  img_array = []
  margin = []
  # song['song_name']=(re.sub("\(",'\n(',song['song_name']))
  offset = 0
  resize_dictionary = {'song_name':{'multiplier':4,'color':(255,255,100),'width':35},
                      'main_artist':{'multiplier':3.5,'color':(255,255,255),'width':50},
                      'feat':{'multiplier':3,'color':(125,143,232),'width':50},
                      'prod':{'multiplier':2.7,'color':(125,143,232),'width':50},
                      'album':{'multiplier':2.5,'color':(153,167,238),'width':60},
                      'date':{'multiplier':2.3,'color':(237,237,237),'width':50}
  }


  if len(l)>5:

    resize_dictionary = {'song_name':{'multiplier':4,'color':(255,255,100),'width':35},
                        'main_artist':{'multiplier':3.5,'color':(255,255,255),'width':50},
                        'feat':{'multiplier':3,'color':(125,143,232),'width':60},
                        'prod':{'multiplier':2.7,'color':(125,143,232),'width':65},
                        'album':{'multiplier':2.5,'color':(153,167,238),'width':60},
                        'date':{'multiplier':2.3,'color':(237,237,237),'width':50}
    }

  # if song['album']=='Single':
  #   resize_dictionary['album']['multiplier']=2.5

  img_array  = []
  img_array1=[]
  fon1 = default
  # res = res*(512/1080)
    # res = 512
  res = int(res*(1-(380/1080)))
  # key = 'song_name'
  g1=VGroup()
  for key in l:
    data = extractor(song[key])
    if data in empty_list:
      continue

    # if key == 'song_name':
    #   data=(re.sub("\(",'\n(',song['song_name']))


    # data = zero_width_cleaner(data)
    width = resize_dictionary[key]['width']
    col = resize_dictionary[key]['color']
    mul = resize_dictionary[key]['multiplier']
    
    # print(res)
    d = limiting_line_space(data,width)

    # img = main_
    lst = ['&\u200b','Featuring\u200b','Released\u200b', 'on\u200b','Produced\u200b', 'by\u200b','On\u200b']

    song_data_color_dictionary = {}
    for words in data.split():
      if words in lst:
        song_data_color_dictionary[strips_words(words,0,1).lower()] = (154,154,154)
      else:
        song_data_color_dictionary[strips_words(words,0,1).lower()] = col
    count = 0

    # display(song_data_color_dictionary)
    # if len(d)>1:
    #   mul = mul*.8
    save_size = ''
    img_array=[]
    
    for line in d:
      
      count+=1

      # display(line)
      # print(markup_texter(line,song_data_color_dictionary,''))
      mrk = markup_texter(line,song_data_color_dictionary,'')
      mrk = mrk.replace('&','&amp;')
      # print(mrk)
      da = MarkupText(mrk,font_size=35).scale(mul)

      # print(markup_texter(line,song_data_color_dictionary))
      
      g1+=da

  if len(g1)>5:
    g1.arrange(DOWN,center=False,aligned_edge=LEFT,buff=0.4)
  elif len(g1)==4:
    g1.arrange(DOWN,center=False,aligned_edge=LEFT,buff=5)
  elif len(g1)==3:
    g1.arrange(DOWN,center=False,aligned_edge=LEFT,buff=10)
  elif len(g1)==5:
    g1.arrange(DOWN,center=False,aligned_edge=LEFT,buff=2.5)
  if k.height == k.width:
    box = Rectangle(  # create a box
            height=k.height*1.1, width=config.frame_width-(k.width)*1.1, fill_color=hexcolor(255,255,100), 
            fill_opacity=0.5, stroke_color=hexcolor(255,255,100)
        )
  else:
    box = Rectangle(  # create a box
            height=k.width*1.1, width=config.frame_width-(k.width)*1.1, fill_color=hexcolor(255,255,100), 
            fill_opacity=0.5, stroke_color=hexcolor(255,255,100)
        )
  g1.scale_to_fit_height(box.height)
  g1.move_to(box)
  g1.move_to(box,aligned_edge=LEFT)
  # g1.scale(0.5)
  timestamps = list(lyr_data.keys())
  count=0
  video_text_sync_data  = []
  print(0,'\u200b','duration:',timestamps[0])
  video_text_sync_data.append(('\u200b',timestamps[0]))
  for item in lyr_data:
    count+=1
    try:
      # print(item,lyr_data[item],'duration:',timestamps[count]-item)
      video_text_sync_data.append((lyr_data[item],timestamps[count]-item))
    except IndexError:
      # print(item,lyr_data[item],'duration:',total_frames-item)
      video_text_sync_data.append((lyr_data[item],total_frames-item))
  # clear()
  g1.arrange(DOWN,center=False,aligned_edge=LEFT,buff=0.15)
  g1.scale_to_fit_height(box.height)
  g1.align_on_border(UP, buff=0.15)
  if g1.width > box.width*.95:
    g1.scale_to_fit_width(box.width*.95)
    g1.move_to(box.get_center())
  g1.next_to(s_art,RIGHT,buff=0.3)
  
  border_sart = k
  # video_text_sync_data
  return c,t,g0,t1,g1,s_art,border_sart,video_text_sync_data

# c,t,g0,t1,g1,s_art,border_sart,video_text_sync_data = elements_for_the_mv(res)
# song_card_v2(res)


In [ ]:
###### FOR LRC METHOD
def step_one():
  # name_of_the_song = done_dl[index]['file_name']
  # fiv_a = card
  
  deezer_lyrics_to_lrc()
  lyr_data = convert_lrc_to_lyr_data()
  times = list(lyr_data)

  # song_duration = int(track['info']['DATA']['DURATION'])
  song_duration = math.ceil(MP3(done_dl[index]['location']).info.length)
  
  total_frames = song_duration*frames
  print('totalframes: ',total_frames,'... possible parts',math.ceil(total_frames/500))

  list_of_lyrics = {'translation':[],
                    'pronounciation':[],
                    'main':[]}


  for i in range(0,total_frames):
    t= ' '
    for item in list_of_lyrics:
      list_of_lyrics[item].append(t)
  # cunt = 0
  for i in times:
    # print(brr)
    # i = int(brr)
    # if cunt%50==0:
    #   time.sleep(1)
    try:
      next=0
      next = i+1
      line = lyr_data[i].strip()
      # display(i,line)

      if line not in ["\u200b","\u200b\u200b",'',"",' '," "] and line not in empty_list:
        # print(next,i,'the line is',line)
        # continue
        # try:
        # cunt+=1
        
        translation,pronounciation = translating(line)
        
        # while (translation,pronounciation) == (False,False):
        #   translation,pronounciation = translating(line.translate(str.maketrans("","",p2)))
        if (translation,pronounciation)==(False,False):
          translation,pronounciation = timer_for_google_translate(line)

        # except TypeError:
        #   continue
        translation, pronounciation = translation.translate(str.maketrans("","",p2)),pronounciation.translate(str.maketrans("","",p2))
        list_of_lyrics['main'][i]=zero_width_cleaner(line,1)
        list_of_lyrics['pronounciation'][i]=zero_width_cleaner(pronounciation,1)
        list_of_lyrics['translation'][i]=zero_width_cleaner(translation,1)

        while next not in times and next < total_frames:
          list_of_lyrics['main'][next]=zero_width_cleaner(line,1)
          list_of_lyrics['pronounciation'][next]=zero_width_cleaner(pronounciation,1)
          list_of_lyrics['translation'][next]=zero_width_cleaner(translation,1)

          
          next+=1
    except IndexError:
      continue


  for i in range(total_frames):
    if list_of_lyrics['main'][i] not in empty_list:
      line = ''
      for item in list_of_lyrics:
        line += list_of_lyrics[item][i]+separator
      line = line.strip(separator)
      # line.split(separator)
      line = line.split(separator)

      for id in range(len(line)):
        line[id]=line[id].strip()
        
      a,b,c=line
      # a = re.sub("[\s\.,]+(?=$)","",a)
      # b = re.sub("[\s\.,]+(?=$)","",b)
      # c = re.sub("[\s\.,]+(?=$)","",c)
      # print(line)
      # display(line)


      # a,b,c=line.split(separator)
      # a,b,c=line.split(separator)
      if norm(b.translate(str.maketrans("","",p2)).lower())==norm(c.translate(str.maketrans("","",p2)).lower()):
        
        if norm(a.translate(str.maketrans("","",p2)).lower())==norm(b.translate(str.maketrans("","",p2)).lower())==norm(c.translate(str.maketrans("","",p2)).lower()):
          list_of_lyrics['translation'][i]=""
          list_of_lyrics['pronounciation'][i]=c
          list_of_lyrics['main'][i]=""
        else:  
        
          list_of_lyrics['main'][i]="" 
          list_of_lyrics['pronounciation'][i]=c
      elif norm(a.translate(str.maketrans("","",p2)).lower())==norm(b.translate(str.maketrans("","",p2)).lower()):
          list_of_lyrics['translation'][i]=""
      elif norm(a.translate(str.maketrans("","",p2)).lower())==norm(c.translate(str.maketrans("","",p2)).lower()):
        list_of_lyrics['translation'][i]=""

      if norm(a.translate(str.maketrans("","",p2)).lower())==norm(c.translate(str.maketrans("","",p2)).lower()):
        list_of_lyrics['translation'][i]=""
        list_of_lyrics['pronounciation'][i]=c
        list_of_lyrics['main'][i]="" 

  for item in lyr_data:
    if lyr_data[item] not in empty_list and lyr_data[item] not in ['\u200b']:
      lyr_data[item]=lyrics_solver(lyr_data[item])
          
  # c=0
  # for item in list_of_lyrics['pronounciation']:
  #   c+=1
  #   print(c,item)
  return list_of_lyrics,total_frames,lyr_data



In [ ]:
# list_of_lyrics = step_one()
def one_frame(res,i,fon1):
  try:
    fiv_a = card
  except NameError:
    fiv_a = Image.open(placeholder_images+song_name+'_song_card_v2.png')
  main = ''
  main = fiv_a
  # i = 345
  # clear()
  # print(i)
  
  main = main.convert('RGBA')
  t = ''
  if lyr_data[i]=="\u200b" or lyr_data[i] in empty_list:
    return main,fon1
  count=0
  for item in list_of_lyrics:
  

    line = list_of_lyrics[item][i]
    if line == '':
      continue
    # h = ['pronounciation','translation','main']
    # item = h[2]
    # print(line)
    if item == 'main':
      t = 1
    if item == 'pronounciation':
      if list_of_lyrics['main'][i]=='' or list_of_lyrics['translation'][i] == '':
        t = 1
    # main = img_array[-1].resize([512,512])
    

    # main = main.convert('RGBA')
    # print(line,item)
    # else:
    # if list_of_lyrics['pronounciation'][i] in [""," ","\u200b"]:
    #   break
    # print('tag',sep,t)
    if item == 'main':
      width = 80
    else:
      width = ''
    count+=1
    # if count==1:
    image,fon1 = text_to_image(line,res,t,fon1,width)
    image = image.convert('RGBA')
    
    # image = line_printer(line,res)
    if image.size[0]>res:
      image = image.resize(iresize_height(res*0.1,image,1),reducing_gap=4)


    image_width = int((res-image.size[0])/2)
    image_height = res-image.size[1]
    up,middle,down = int(res*0.47-image.size[1]),int(image_height/1.6),int(res*0.95-image.size[1])
    

    if item == 'pronounciation':
      main.alpha_composite(image,(image_width,middle))
    if item == 'translation':
      main.alpha_composite(image,(image_width,down))
    if item =='main':
      main.alpha_composite(image,(image_width,up))

  return main,fon1


In [ ]:
def text_to_image(captions,res,types,fon1,width):
  # a = re.sub('\n\n\n','\u200b',captions)
  # a = re.sub('\n\n','\u200b',a)
  # a = re.sub('\n',' ',a).strip()
  # a = re.sub('\u200b','\n',captions)

  # if width not in empty_list:
  #   brrt = int(res*0.06)
  # else:
  color = (154,154,154)
  #   if len(captions)<int(res*0.1):
  #     brrt = int(res*0.06)
  #   else:
  #     brrt = int(res*0.1)

  # brrt = int(res*0.09)
  
  # if len(captions)>80:
  #   brrt = 50
  # else:
  #   brrt = 40
  # if len(captions)>60:
  #   brrt = int(len(captions)/1.67)
  #   if brrt>45:
  #     brrt = int(len(captions)/2)
  #   if brrt<40:
  #     brrt = 40
  #   # print(brrt)
  # else:
  #   brrt = 40
  brrt = 40
  # print('width',brrt)
  lines = ''
  line= ''

  # captions = captions.split('\n')
  # if len(captions) > brrt:
  #   w = textwrap.TextWrapper(width=brrt,break_long_words=False,drop_whitespace=False,break_on_hyphens=False)
  #   lines ='\n'.join(w.wrap(captions))
  # else:
  #   lines = captions
  lines = limiting_line_space(captions,brrt)
  img_array = []
  margin = []
  offset = 0
  # fon1 = default
  c = 0
  for line in lines:
    if line not in empty_list or line!='':
      c+=1
      # if c==1:
      #   blank = printer(line,'',(255,255,255,0),'','','')
      #   blank = blank.resize(iresize_height(res*0.1,blank,1))

      if types == 1:
        # display(line)
        im,fon1,size = main_printer(line,fon1,color,1,1,color_dictionary,"",1)
        
        # im = im.resize(iresize_height(res*0.1,im,1))

        
        
        # save_size = im.size
        
      else:
        color= (154,154,154)
        cd = {}
        for words in line:
          cd[words]=(154,154,154)
        im,fon1,size = main_printer(line.strip(),default,color,1,1,cd,'',1)
        # display(im)
      # im = im.resize(iresize_height(res*0.1,im,1))
      
      

      # display(im)
      # if c>1:
        
      #   im = im.resize(iresize_height(save_size[1],im,1))
        # im = im.resize(iresize_height(res*0.1,im,1))
        # im = im.resize(iresize_height(img_array[-1].size[1],im,1))
      
      
      marg,off = im.size

      img_array.append(im)
      margin.append(marg)
      offset+=off
    
  dim = max(margin),offset
  # print(dim)
  main = Image.new('RGBA',dim,(0,0,0,0))
  main = main.convert('RGBA')
  # main = main
  margin = offset = 0
  for im in img_array:
    margin = 0
    margin=int((main.size[0]-im.size[0])/2)
    # print(im.size)
    main.alpha_composite(im,(margin,offset))

    offset+=im.size[1]

  if main.size[0]>res:
    main = main.resize(iresize(res,main,1),reducing_gap=4)
  if main.size[1]>res*c*0.1:
    main = main.resize(iresize_height(res*c*0.1,main,1),reducing_gap=4)
  
  # return main
  return main,fon1

# test(884)

In [ ]:
# makes_video()

In [ ]:
def test(i):
  
  l=[]
  for item in lyr_data:
    if lyr_data[item]!="\u200b":
      l.append(item)

  if i in empty_list:

    i = l[int(len(l)/2)]
  
  if i not in l:
    a= count = i
    

    while count not in l:
      if count>max(l):
        count-=1
      else:
        count+=1
      
    i= count
    print(a,'was not in data. This is for timestamp',i,'instead.\n')
  else:
    print('Image for timestamp',i,'\n')
  fon1 = default
  # print(i)
  for item in list_of_lyrics:
    print(item,list_of_lyrics[item][i])
  
  display(one_frame(res,i,fon1)[0])
  return i
  


In [ ]:
def output_video(frames,folder_name,j,img_array):
  import numpy as np
  import cv2
  from cv2 import cvtColor,COLOR_BGR2RGB
  

  im1 = cvtColor(np.array(img_array[0].convert('RGB')),COLOR_BGR2RGB)
  height, width, layers = im1.shape
  size = (width, height)
  out = cv2.VideoWriter(folder_name+str(j)+'.mp4',cv2.VideoWriter_fourcc(*'DIVX'), frames, size)
  cv2_im_array = []
  for image in img_array:
    image.convert('RGB')
    image = image.convert('RGB')
    im = cvtColor(np.array(image),COLOR_BGR2RGB)
    cv2_im_array.append(im)

  for i in range(len(cv2_im_array)):
    out.write(cv2_im_array[i])

  out.release()

from moviepy.editor import *
def combines_video(folder_name,index):

  update_dl()
  # done_dl = to_json({},dmd,'read')
  
  video_file_name = folder_remove(folder_name,0,['.mp4'],0)

  video_array =[]

  for i in range(video_file_name):
    # print(folder_name+str(i+1)+'.mp4')
    video_array.append(VideoFileClip(folder_name+str(i)+'.mp4',fps_source='fps'))


  final_clip = concatenate_videoclips(video_array)
  # file_name = folder_name+song_name+' - '+fav_key+' - '+pattern+'.mp4'
  file_name = song_name+'.mp4'
  # try:
  sound = AudioSegment.from_file(done_dl[index]['location'])
  sound.export("/content/temp_music.mp3", format="mp3", bitrate="128k")
  time.sleep(1)
  audio = AudioFileClip('/content/temp_music.mp3')
  
  
  vid_and_audio = final_clip.set_audio(audio.audio_fadein(0.1).audio_fadeout(1.5))  
  
  

  if os.path.exists(file_name):
    os.remove(file_name)

  # file_name_string = t[0]+'.mp4'
  vid_and_audio.write_videofile(folder_name+file_name,audio_codec='libmp3lame',preset='slow')
  shutil.move(folder_name+file_name,key_destination+file_name)
  clear()
  folder_remove(folder_name,1,['.mp4'],0)
  print(file_name,'saved')


def folder_remove(folder_name,delete,extension,show):
  i=0

  string = ['counted','deleted']
  
  for ext in extension:
    i=0
    if show == 1:
      print(ext,'files: ')

    for files in os.listdir(folder_name):
      # t = ''
      if files.endswith(ext):
        t= ''
        if len(files.split('by'))==1 and len(files.split(ext)[0].split())==1:
          # print(files)
          try:
            t = int(files.split(ext)[0])
          except ValueError:
            t = 'no'

          i+=1
          if show == 1 and t not in ['no', '']:
            print(files)
          if delete == 1 and t not in ['no', '']:
            os.remove(folder_name+files)
        else:
          if show == 1 and t not in ['no', '']:
            print(files)
        
    print(string[delete],str(i),ext,'files.')
  return i

# makes_video()
# combines_video()

In [ ]:
def play_video(version):
  if version in empty_list:
    return Video(key_destination+done_dl[index]['file_name'].replace('\u200b','')+'.mp4',embed=True)
  else:
    return Video(key_destination+done_dl[index]['file_name'].replace('\u200b','')+'.mp4',embed=True)
def makes_video():
  img_array = []
  number = 0
  fon1 = default
  # res = 1080
  # main = fiv_a_maker(res,song_name)
  main = card
  w= ""
  for i in range(total_frames):
    
    if i in list(lyr_data.keys()):
      
      
      main,fon1 = one_frame(res,i,fon1)
    # else:
    #   main = 
    ##############################################################
      # if lyr_data[i] not in ['\u200b']:
      #   w+= lyr_data[i]+" "
        
      #   h = matrix(w.strip())
        
      # if lyr_data[i] in ['\u200b']:
      #   main.alpha_composite(h,(0,0))
      # else:
      #   img_array.append(main)
    ###########################################################
    img_array.append(main)
    # display(main)
    # clear()
    
    if len(img_array)==500:
      output_video(30,video_path,number,img_array)
      clear()
      print('Creating',song_name+'...')
      print('saved',number)
      
      number+=1
      img_array=[]

  if img_array != []:
    output_video(30,video_path,number,img_array)
    number+=1
    print('Creating',song_name+'...')
    print('saved',number)
    img_array=[]
  clear()
  # song_name = ''
  combines_video(video_path,index)
  for files in os.listdir(placeholder_images):
    os.remove(placeholder_images+files)
  os.remove('/content/temp_music.mp3')

  print('Video: ',key_destination+done_dl[index]['file_name']+'.mp4')


In [ ]:

def deezer_song_data_for_desparate_times():
  # if len(to_json({},'deezer_appendix','read')[int(index)-1].split(separator))==3:
  if 'deezer' not in get_sd()[index]['link'].keys():
    spotify_song_data_for_desparate_times()
    return 'Data collected from Spotify'
  else:
    song = to_json({},sid,'read')
    a = track['tags']
    song_name = a['title']
    main_artist = a['artist']
    album = a['album']
    tracknumber = a['tracknumber'].split('/')[0]
    feat = prod = ''
    date = a['date']
    lyrics = deezer_lyrics_splitter(lyrics_sync,split_lyrics)[0]

    if tracknumber == None:
      album = 'On\u200b'+' '+album
    if tracknumber is not None:
      if a['tracknumber'] in ['1/1']:
        album = 'Single'
      else:
        album = '#'+str(tracknumber)+' on\u200b '+album

    if date is not None:
      date = datetimedate.fromisoformat(date)

      date = "Released\u200b on\u200b "+date.strftime("%B "+num_to_ordinal(date.day)+", %Y")

    album_art = song_art = song_id = genius_link = ''
    song[index]={}
    song[index]={
        'song_name':song_name,
        'feat':feat,
        'prod':prod,
        'date':date,
        'album':album,
        'cover_art':album_art,
        'song_art':song_art,
        'main_artist':main_artist,
        'lyrics':lyrics,
        'song_id':song_id,
        'url':genius_link,
        'mp3_file_index':index
    }

    to_json(song,sid,'write')

    return song[index]




In [ ]:
# for item in to_json({},slsd,'read').keys():
#   print(item)
def rewrite_slsd():
  h = to_json({},slsd,'read')
  # h1 = list(h.keys())
  for item in h:
    if norm(item) == norm(song_name):
      d= item
    

  h.pop(d)

  to_json(h,slsd,'write')

In [ ]:
def select_color_dictionary(lyrics):
  
  time.sleep(2)
  clear()
  yes = ''
  while yes in empty_list:
    print('Making color palette...\n')
    im,dic = see_color_palette(5,colors(lyrics))
    display(im)

    # print('word frequency\n')
    color_frequency_bubble(lyrics,dic)
    
    # h = matrix(lyrics,'',dic)
    # print('\nSelf-similarity')
    # display(h.resize([512,512]))
    
    time.sleep(2)
    beep()
    yes = input("\nTo select, this I need 1: \n")
    
    clear()
  color_saver(index,dic)
  return dic



In [ ]:
def play_another_video(s):
  l = []
  video_path = key_destination
  count=0
  if s in ["",''," ",' ']:
    video_list = (os.listdir(video_path))
  if s==1:
    video_list = sorted(os.listdir(video_path))

  for item in video_list:
    count+=1
    print(str(count)+sep+str(count-len(video_list))+'. ',item)

    l.append(video_path+item)
  time.sleep(1)
  choice = input('\nVideo to play: ')
  if choice in ["",''," ",' ']:
    clear()
    return 'Oka bai, see ya later'
  choice = int(choice)
  clear()
  print('Playing',l[choice-1].split(video_path)[-1][:-4]+'....'+'\n')
  if getfilesize(l[choice-1],2) > 21*1024**2:
    if os.path.exists(placeholder_images) is False:
      os.mkdir(placeholder_images)

    new_vid =placeholder_images+l[choice-1].split(video_path)[-1]
    com_file = os.path.splitext(new_vid)[0]+'_compressed.mp4'
    if os.path.exists(com_file) is False:
        
      shutil.copy(l[choice-1],new_vid)
      print(getfilesize(l[choice-1],1)+' video too big to display.')
      print('Compressing...')
      compress_video(new_vid,21*1024,True,'_compressed')
      data_saved = format_size(getfilesize(l[choice-1],2)-getfilesize(com_file,2))
      print('Compressed video is '+getfilesize(com_file,1)+' ['+data_saved+' saved]')
      print('\n')
      os.remove(new_vid)
    # time.sleep(2)
    clear()
    print('Playing',l[choice-1].split(video_path)[-1][:-4]+'....'+'\n')
    time.sleep(2)
    display(Video(com_file,embed=True))
  else:
    display(Video(l[choice-1],embed=True))

  

In [ ]:
def line_splitter(line):
  w= ''
  list_of_words= fontaine(line)
  for i in range(len(list_of_words)):
    word,font = list_of_words[i]['word'],list_of_words[i]['font']
    # display(word)
    # im = printer(word,font,color,1,color_dictionary)
    w+=word
    # img_array.append(im)
    
    if i <len(list_of_words):
      try:
        if list_of_words[i+1]['word'][0] not in non_space_punct and font not in sa:      
          # img_array.append(blank_space)
          w+=' '
      except IndexError:
        break
  return  w
  # show()

In [ ]:

def spotify_song_data_for_desparate_times():
  # link = to_json({},'deezer_appendix','read')[int(index)-1].split(separator)[1]
  song = to_json({},sid,'read')
  sd = get_sd()
  if 'spotify' not in sd[index]['link']:
    if 'soundcloud' in sd[index]['link']:
      CLIENT_ID,api=soundcloud_token_checker()
      sc_track = api.resolve(sd[index]['link']['soundcloud'])
      song_art = album_art = sc_track.artwork_url.replace('-large','-t500x500')
  else:
    spotify = spotify_token_checker()
    data = spotify.track(sd[index]['link']['spotify'])
    song_art = album_art = data['album']['images'][0]['url']  

  sd = get_sd()[index]
  
  
  name = sd['song_name']
  album = sd['album_name']
  feat = prod = ''
  artist = sd['artist']
  tracknumber = sd['track_position']
  date = sd['album_release_date']
  
  lyrics = song_id = genius_link = ''

  if album not in ['Single']:
    
    album = '#'+tracknumber.split('/')[0]+' on\u200b '+album

  if date not in empty_list:
    date = 'Released\u200b on\u200b '+date
  
  if index not in song:
    song[index]={}

  # song[index]={
  #   'song_name':name,
  #   'feat':feat,
  #   'prod':prod,
  #   'date':date,
  #   'album':album,
  #   'cover_art':album_art,
  #   'song_art':song_art,
  #   'main_artist':artist,
  #   'lyrics':lyrics,
  #   'song_id':song_id,
  #   'url':genius_link,
  #   'mp3_file_index':index
  # }
  song[index]['song_name']=name
  song[index]['feat']=feat
  song[index]['prod']=prod
  song[index]['date']=date
  song[index]['album']=album
  song[index]['cover_art']=album_art
  song[index]['song_art']=song_art
  song[index]['main_artist']=artist
  song[index]['lyrics']=lyrics
  song[index]['song_id']=song_id
  song[index]['url']=genius_link
  song[index]['mp3_file_index']=index
  to_json(song,sid,'write')

  return song[index]

In [ ]:
def done_videos():
  done_videos = []
  for files in os.listdir(key_destination):
    done_videos.append(files.split('.mp4')[0])

  return done_videos

In [ ]:
def lyrics_from_slsd(end_string):
  if end_string in empty_list:
    end_string = '\n'
  w=''
  for item in to_json({},slsd,'read')[index].split('\n'):
    c,d = item.split(']')
    if d not in ["\u200b"] and item not in empty_list:
      
      w+=d+end_string

  return w.strip()
# alternate_lyrics((1))
# lyrics_from_slsd('\\n')

In [ ]:
def songs_in_slsd():
  n = []
  for item in to_json({},slsd,'read').keys():
    n.append(done_dl[item]['file_name']+sep+item)
  c=  0
  n = sorted(n)
  for i in range(len(n)):
    c+=1
    print(c,sep,n[i].split(sep)[-1],sep,n[i].split(sep)[0])

def show():
  count=1
  for i in lyr_data:
    if lyr_data[i] not in ["\u200b"] and lyr_data[i] not in empty_list:
      print(count,sep,i,sep,lyr_data[i])
      count+=1



In [ ]:
def lyr_data_to_lyrics():
  w=''
  for i in lyr_data:
    if lyr_data[i] not in ["\u200b"]:
      p = lyr_data[i].strip()+'\n'
      w += lyrics_solver(p)+'\n'
  return w.strip()

# lyr_data_to_lyrics()

In [ ]:
def musixmatch_lyrics_from_sync_data():
  print('Copy and put "\\n before". Go to this — https://pinetools.com/add-text-each-line\n')
  g= input('The lyrics: \n\n')
  w=''
  c = 0
  clear()
  for item in g.split('\\n'):
    c+=1
    if item not in empty_list:
      if c%2==0:
        line = item.strip()
        w+=line+'\n'
  return w.strip()


In [ ]:
# line = 
# strp = "sofuhn يلا حبي يلا jjjjjjjj مش حلو 😜 تجرحني بهواك প্রতিটি জানালায় 爱 bts হাসিমুখ bts biot   "
# strp = '爱? sofuhn'
# strp = 'يلا حبي يلا jjjjjjjj'

def fontained(line):
  
  

  fon1 = default
  # regular_files = font_file_list()
  regular_files.insert(0,default)
  # regular_files = font_files
  

  w=''
  l = []
  for word in line.split():
    # w+='\u200b'
    for char in word:
      # print(allowed_punct)
      if char in ["\u200d"]:
        l.append({'letter':char,'font':fon1})
        continue

      if char in ['-',',','.']:
        l.append({'letter':char,'font':fon1})

        continue
      if char in allowed_punct:
        # print(char,allowed_punct)
          # l.append({'letter':char,'font':fon1})
        # print(char)
        
        l.append({'letter':char,'font':default})
        continue
      if check_font(font_path+fon1,char):
        if check_font(font_path+default,char):
          fon1 = default
          # w=char
          l.append({'letter':char,'font':fon1})
        else:
          l.append({'letter':char,'font':fon1})
          

      else:
        for files in regular_files:
          if check_font(font_path+files,char):
            fon1 = files
            regular_files.insert(1,files)
            break
        l.append({'letter':char,'font':fon1})
    l.append({'letter':' ','font':fon1})
  # print(l)
  p = []
  w=''
  font = default
  for i in range(len(l)):

    if l[i]['font']==font:
      w+=l[i]['letter']
    else:
      if w not in empty_list:
        p.append({'word':w.strip(),'font':font})
      w=l[i]['letter']

      font = l[i]['font']
  p.append({'word':w.strip(),'font':font})
  return p



In [ ]:
############# VIDEO COMPRESSOR FROM GITHUB
import ffmpeg
def compress_video(video_full_path, size_upper_bound, two_pass=True, filename_suffix='1'):
    """
    Compress video file to max-supported size.
    :param video_full_path: the video you want to compress.
    :param size_upper_bound: Max video size in KB.
    :param two_pass: Set to True to enable two-pass calculation.
    :param filename_suffix: Add a suffix for new video.
    :return: out_put_name or error
    """
    filename, extension = os.path.splitext(video_full_path)
    extension = '.mp4'
    output_file_name = filename + filename_suffix + extension

    total_bitrate_lower_bound = 11000
    min_audio_bitrate = 32000
    max_audio_bitrate = 256000
    min_video_bitrate = 100000

    try:
        # Bitrate reference: https://en.wikipedia.org/wiki/Bit_rate#Encoding_bit_rate
        probe = ffmpeg.probe(video_full_path)
        # Video duration, in s.
        duration = float(probe['format']['duration'])
        # Audio bitrate, in bps.
        audio_bitrate = float(next((s for s in probe['streams'] if s['codec_type'] == 'audio'), None)['bit_rate'])
        # Target total bitrate, in bps.
        target_total_bitrate = (size_upper_bound * 1024 * 8) / (1.073741824 * duration)
        if target_total_bitrate < total_bitrate_lower_bound:
            print('Bitrate is extremely low! Stop compress!')
            return False

        # Best min size, in kB.
        best_min_size = (min_audio_bitrate + min_video_bitrate) * (1.073741824 * duration) / (8 * 1024)
        if size_upper_bound < best_min_size:
            print('Quality not good! Recommended minimum size:', '{:,}'.format(int(best_min_size)), 'KB.')
            # return False

        # Target audio bitrate, in bps.
        audio_bitrate = audio_bitrate

        # target audio bitrate, in bps
        if 10 * audio_bitrate > target_total_bitrate:
            audio_bitrate = target_total_bitrate / 10
            if audio_bitrate < min_audio_bitrate < target_total_bitrate:
                audio_bitrate = min_audio_bitrate
            elif audio_bitrate > max_audio_bitrate:
                audio_bitrate = max_audio_bitrate

        # Target video bitrate, in bps.
        video_bitrate = target_total_bitrate - audio_bitrate
        if video_bitrate < 1000:
            print('Bitrate {} is extremely low! Stop compress.'.format(video_bitrate))
            return False

        i = ffmpeg.input(video_full_path)
        if two_pass:
            ffmpeg.output(i, '/dev/null' if os.path.exists('/dev/null') else 'NUL',
                          **{'c:v': 'libx264', 'b:v': video_bitrate, 'pass': 1, 'f': 'mp4'}
                          ).overwrite_output().run()
            ffmpeg.output(i, output_file_name,
                          **{'c:v': 'libx264', 'b:v': video_bitrate, 'pass': 2, 'c:a': 'aac', 'b:a': audio_bitrate}
                          ).overwrite_output().run()
        else:
            ffmpeg.output(i, output_file_name,
                          **{'c:v': 'libx264', 'b:v': video_bitrate, 'c:a': 'aac', 'b:a': audio_bitrate}
                          ).overwrite_output().run()

        if os.path.getsize(output_file_name) <= size_upper_bound * 1024:
            return output_file_name
        elif os.path.getsize(output_file_name) < os.path.getsize(video_full_path):  # Do it again
            return compress_video(output_file_name, size_upper_bound)
        else:
            return False
    except FileNotFoundError as e:
        print('You do not have ffmpeg installed!', e)
        print('You can install ffmpeg by reading https://github.com/kkroening/ffmpeg-python/issues/251')
        return False

In [ ]:
def adds_to_font_database(line):
  k = to_json({},fd,'read')
  w=''
  for word in line.split():
    try:
      if k[word] != empty_list:
        w+=word+' '
        continue
    except KeyError:
      a= fontained(word)
      for i in range(len(a)):
        w+=a[i]['word']+' '
        k[a[i]['word']]=a[i]['font']
      to_json(k,fd,'write')
      clear()
  return w.strip()

# k = to_json({},fd,'read')
# k = adds_to_font_database(line)

def fontaine(line):
  line = adds_to_font_database(line)
  k = to_json({},fd,'read')
  w= ''
  l=[]
  font = default
  for word in line.split():
    if k[word]==font:
      w+=word+' '
    else:
      if w not in empty_list:
        l.append({'word':w.strip(),'font':font})
      w=word+' '
      font = k[word]
  l.append({'word':w.strip(),'font':font})
  return l

  


In [ ]:
from manim import *
config.frame_height=config.frame_width
def song_card_v2(res):
  os.chdir('/content/media')
  
  class song_card_v2(Scene):
    
      def construct(self):

        
        # try:
        # c,t,g0,t1,g1,s_art,border_sart,video_text_sync_data = elements_for_the_mv()
        self.add(c,t,g0,t1,g1,s_art,border_sart)
        # except NameError:
        #   print('Going to COLLECT DATA')
        #   c,t,g0,t1,g1,s_art,border_sart,video_text_sync_data = elements_for_the_mv()
        #   self.add(c,t,g0,g1,s_art,border_sart)
        
        
          #   self.add(item)
  %manim -qm -r $res,$res -v WARNING song_card_v2
  for files in os.listdir('/content/media/media/images/media/'):
    if files.startswith('song_card_v2_'):
      # os.rename('/content/media/media/images/media/song_card_v2_ManimCE_v0.14.0.png',placeholder_images+song_name+'_song_card_v2.png')
      os.rename('/content/media/media/images/media/'+files,placeholder_images+song_name+'_song_card_v2.png')
  # os.chdir('/content')
# song_card_new(600)
# clear()

# from manim import *
# config.frame_height=config.frame_width
def intro_video(res,frame):
  os.chdir('/content/media')
  class la_musica(Scene):
    
    def construct(self):
        
      # c,t,g0,t1,g1,s_art,border_sart,video_text_sync_data = elements_for_the_mv()
      # self.add(g[102])
      self.play(FadeIn(c),FadeIn(t),Write(g0),FadeIn(t1),TransformFromCopy(c,s_art))
      self.play(Create(border_sart),Write(g1))
      
  if frame in empty_list:
    frame = 30

  %manim -qh -r $res,$res --disable_caching -v WARNING la_musica 
  vid = '/content/media/media/videos/media/'+str(res)+'p'+str(frame)+'/la_musica.mp4'
  des = video_path+song_name+'_intro_video.mp4'
  os.rename(vid,des)

  # os.chdir('/content')


# intro_video(600,'')

# from manim import *
# from manim import *
# config.frame_height=config.fra sme_width
def makes_video_v2(res,frame,compressor):
  
  os.chdir('/content/media')
  shutil.rmtree('/content/media/media/videos/media/')
  sound = AudioSegment.from_file(done_dl[index]['location'])
  sound.export("/content/SYNC_MUSIC/"+done_dl[index]['file_name']+"_128k.mp3", format="mp3", bitrate="128k")
  sound_128k = "/content/SYNC_MUSIC/"+done_dl[index]['file_name']+"_128k.mp3"
  class lala_musica(Scene):
    def construct(self):
      # song_card_v2(res)
      # clear()
      self.lag_ratio=0    
      # offset = 0
      # shutil.copy(done_dl[index]['location'],'music.mp3')
      # song_card_v2(res)
      # self.wait(0)
      black = ImageMobject(Image.new('RGB',[res,res],(0,0,0))).scale_to_fit_width(config.frame_width)
      
      
      backdrop = ImageMobject(placeholder_images+song_name+'_song_card_v2.png')
      # video_text_sync_data[0][1]/30
      if video_text_sync_data[0][0] not in ['\u200b','\u200b'] or video_text_sync_data[0][1]/30 <2:
        print('woahh, this song doesnt waste time...')
        self.play(FadeIn(c),FadeIn(t),Write(g0),FadeIn(t1),TransformFromCopy(c,s_art),run_time=1)
        self.play(Create(border_sart),Write(g1),run_time=1)
        # self.wait(0)
        # self.add(backdrop.scale_to_fit_width(config.frame_width))
        
        self.remove(c,t,g0,t1,s_art,border_sart,g1)
        self.wait(0)
        self.add(backdrop.scale_to_fit_width(config.frame_width))
        
        api_caller3=1
        self.wait(0)
        self.add_sound(sound_128k)
        offset = 2
      
      count = 0
      for item in video_text_sync_data:
        # print(count)
        
        line,duration = item
        # count+=1
        print(count,sep,line,sep,duration)
        duration = duration/30
        
        if count%20==0:
          clear()
          print('Making '+song_name+'....')
          print(str(count)+'/'+str(len(video_text_sync_data)-1),sep,line)
        # = 0.999999999999999999
        
          # self.add(g0)
          
        if count==0:
          try:
            if api_caller3==1:
              print('INTRO DONE')
          except NameError:
            # os.chdir('/content')
            print('DAMNNNN THIS SONG HAS A LONG ASS INSTRUMENTAL NOTHING')
            # self.add(black)
            self.wait(0.1)
            self.add_sound(sound_128k)
            self.play(FadeIn(c),FadeIn(t),Write(g0),FadeIn(t1),TransformFromCopy(c,s_art),run_time=duration*0.5)
            self.play(Create(border_sart),Write(g1),run_time=duration-duration*0.5)
            self.remove(c,t,g0,t1,s_art,border_sart,g1)
            self.wait(0)           
            self.add(backdrop.scale_to_fit_width(config.frame_width))
            # # self.wait(0)
            # self.add(backdrop.scale_to_fit_width(config.frame_width))
            # self.remove(c,t,g0,t1,s_art,border_sart,g1)
            # self.wait(duration)
            api_caller3=1
            count+=1
            continue

        
        if count== len(video_text_sync_data)-1:
          
          if deletion == 1:
              # self.remove(g)
            # self.play(FadeOut(g,scale=0.98),run_time=duration*.999*.2,rate_func=linear,lag_ratio=0)    
            self.remove(g)
          if deletion == 2:
            # self.remove(g,k)
            # self.play(FadeOut(g,scale=0.98),FadeOut(k,scale=0.98),run_time=duration*.999*.2,rate_func=linear,lag_ratio=0)   
            self.remove(g,k)
          if deletion == 3:
            # self.remove(g,h)
            # self.play(FadeOut(g,scale=0.98),FadeOut(h,scale=0.98),run_time=duration*.999*.2,rate_func=linear,lag_ratio=0)   
            self.remove(g,h)
          if deletion == 4:
            # self.remove(g,h,k)
            # self.play(FadeOut(g,scale=0.98),FadeOut(h,scale=0.98),FadeOut(k,scale=0.98),run_time=duration*.999*.2,rate_func=linear,lag_ratio=0)   
            self.remove(g,h,k)

          if duration > 1:
            self.play(FadeIn(black),run_time=duration)
          else:            
            self.play(FadeIn(black),run_time=1)
          # self.add(black)
          break            

        top_text,middle_text,bottom_text = streamlines_lines(line)
        # count+=1
        #H - UP G - MIDDLE K - DOWN
        #W - UP X - MIDDLE - Y - DOWN
        
        w = ''
        for item in limiting_line_space(top_text,40):
          w+=markup_texter(item,color_dictionary,1)+'\n'
        
          # w=w.replace("&","&amp;")

        x = ''
        if top_text != '':
          middle_text_width = 45
        else:
          middle_text_width = 40
        for item in limiting_line_space(middle_text,middle_text_width):
          if top_text != '':
            x+=markup_texter(item,'',1)+'\n'
          else:
            x+=markup_texter(item,color_dictionary,1)+'\n'
            # x=x.replace("&","&amp;")

        y = ''
        for item in limiting_line_space(bottom_text,45):
          y+=markup_texter(item,'',1)+'\n'
          # y=y.replace("&","&amp;")
        # duration = round(duration,1)

        
        
        if line in ['\u200b','\u200b\u200b'] or line in empty_list:

          if count>1:
            # # if video_text_sync_data[count-1] not in ['\u200b']:
            # if deletion == 1:
            #   # self.remove(g)
            #   # self.play(FadeOut(g,scale=0.98),run_time=duration*.999*.19,rate_func=linear,lag_ratio=0)    
            #   self.remove(g)

            # if deletion == 2:
            #   # self.remove(g,k)
            #   # self.play(FadeOut(g,scale=0.98),FadeOut(k,scale=0.98),run_time=duration*.999*.19,rate_func=linear,lag_ratio=0)   
            #   self.remove(g,k)

            # if deletion == 3:
            #   # self.remove(g,h)
            #   # self.play(FadeOut(g,scale=0.98),FadeOut(h,scale=0.98),run_time=duration*.999*.19,rate_func=linear,lag_ratio=0)   
            #   self.remove(g,h)

            # if deletion == 4:
              # self.remove(g,h,k)
              # self.play(FadeOut(g,scale=0.98),FadeOut(h,scale=0.98),FadeOut(k,scale=0.98),run_time=duration*.999*.19,rate_func=linear,lag_ratio=0)   
            self.remove(g,h,k)
            
          # if deletion == 0:
          #   self.wait(du)
          #   self.wait(duration*.999)
          #   # deletion = 0
          # elif count>1 and deletion == 0:
          #   # print('repeater')
          #   self.wait(duration*.999)
          
          self.wait(duration)
          #   continue
          # else
          count+=1
          continue
          # continue
        count+=1
          # continue
          # if count==1:   
          #   print(count,'FIRST')
          #   self.wait(duration)
          
          # continue
        
        g=VGroup()
        h = VGroup()
        k= VGroup()
        for lines in y.split('\n'):
          text = MarkupText(lines.replace('&','&amp;'))
          k+=text
        k.arrange(DOWN)

        for lines in x.split('\n'):
          text = MarkupText(lines.replace('&','&amp;'))
          g+=text
        g.arrange(DOWN)

        for lines in w.split('\n'):
          text = MarkupText(lines.replace('&','&amp;'))
          h+=text
        h.arrange(DOWN)
        #H - UP G - MIDDLE K - DOWN
        #W - UP X - MIDDLE - Y - DOWN
        g.align_on_border(DOWN,buff=5.5).scale(1.6)
        h.align_on_border(UP,buff=4.5).scale(1.6)
        k.align_on_border(DOWN,buff=1.5).scale(1.6)

        if g.width>config.frame_width*0.95:
          g.scale_to_fit_width(config.frame_width*0.95)
        if h.width>config.frame_width*0.95:
          h.scale_to_fit_width(config.frame_width*0.95)
        if k.width>config.frame_width*0.95:
          k.scale_to_fit_width(config.frame_width*0.95)
        # print(duration)
        # precision = 1
        # self.remove(g,h,k)
        # self.add(g,h,k)
        duration = duration
        fir,sec,las = duration*0.1,duration*0.9,duration*0.1
        deletion = 0
        if top_text == '' and bottom_text == '':
          deletion = 1
          self.play(Write(g,rate_func=linear),run_time=fir,rate_func=linear,lag_ratio=0)  

        if top_text == '' and bottom_text != '':
          deletion = 2
          self.play(Write(g,rate_func=linear),Write(k,rate_func=linear),run_time=fir,rate_func=linear,lag_ratio=0)

        if top_text != '' and bottom_text =='':
          deletion = 3
          self.play(Write(g,rate_func=linear),Write(h,rate_func=linear),run_time=fir,rate_func=linear,lag_ratio=0)

        if top_text != '' and bottom_text !='':
          deletion = 4
          self.play(Write(h,rate_func=linear),Write(g,rate_func=linear),Write(k,rate_func=linear),run_time=fir,rate_func=linear,lag_ratio=0)
      
        # fir,sec,las = round(fir,1),round(sec,1),round(las,1)
        # if count==0:
        #   self.play(Write(g),run_time=fir,la)
        self.lag_ratio=0
        # self.play(Write(g),Write(h),Write(k),run_time=fir,rate_func=linear,lag_ratio=0)
        self.wait(duration-fir)
        # self.remove(g)
        # self.play(FadeOut(g,scale=0.98),run_time=las,rate_func=linear)
        # self.remove(g)
        
          
  clear()
  clear()
  # os.chdir('/content')
  if frame in empty_list:
    frame = 30

  %manim -qm -r $res,$res -v WARNING lala_musica
  clear()
  clear()
  vid = '/content/media/media/videos/media/'+str(res)+'p'+str(frame)+'/lala_musica.mp4'
  des = key_destination+song_name.replace('\u200b','')+'.mp4'
  try:
    vid_file_size = round(os.path.getsize(vid)/1024/1024,1)
  except Exception as e:
    
    vid = '/content/media/media/videos/media/'+str(res)+'p'+'60'+'/lala_musica.mp4'
    des = key_destination+song_name.replace('\u200b','')+'.mp4'
    vid_file_size = round(os.path.getsize(vid)/1024/1024,1)

  if compressor == '1':
  
    
    # vid_file_size
    if vid_file_size>21:
      print('FILE SIZES GETTING OUT OF HAND ['+str(vid_file_size)+'MB]!!!\nCompressing '+song['song_name']+'...')
      compress_video(vid,21*1024,True,'_compressed')
      new_vid = '/content/media/media/videos/media/'+str(res)+'p'+str(frame)+'/lala_musica_compressed.mp4'
    else:
      print('FILE SIZE IS A TOLERABLE '+str(vid_file_size)+'MB!!!')
      new_vid = vid
    
    shutil.move(new_vid,des)
  else:
    shutil.move(vid,des)
  os.remove(sound_128k)
  time.sleep(2)
  clear()
  print('Music video '+song_name+' created.\nFile size: '+getfilesize(des,1))
  # clear()
# os.chdir('/content/')
# song_card_v2(600)

# os.getcwd()

In [ ]:
def matrix(string,alpha,color_dictionary):
  # w=''
  l = []
  string  = string.split()
  for words in string:
    if words not in empty_list:
      l.append(words)
  string = l
  if len(string) <= res:
    
    mult = math.floor(res/len(string))
  else:
    # res1 = len(string)
    mult = 1

  pixel = mult
  width = height = len(string)*mult
  if alpha == 1:
    main_image = Image.new('RGBA',[width,height],(0,0,0,0))
  else:
    main_image = Image.new('RGBA',[width,height],(0,0,0))

  margin=offset = 0

  for word_a in string:
    for word_b in string:
      if word_a not in empty_list and word_b not in empty_list:
        displ
        if strips_words(word_a.strip().lower(),0,1)==strips_words(word_b.strip().lower(),0,1) and word_a not in p1 and word_a not in p2:
          
          main_image.paste(Image.new('RGB',[pixel,pixel],color_dictionary[strips_words(word_a.strip().lower(),0,1)]),(margin,offset))
          offset+=pixel
        else:
          offset+=pixel
    margin+=pixel
    offset=0
  # print(lyr_data[k])
  return main_image.resize([res,res],reducing_gap=4)

In [ ]:
def chinese_and_japanese(line):
    # line='あなぐら ぐ ら し の 渦 の 中'
  # line ='手を伸ばしてさあさあさあ歩きだしてぷくぷくぷく'
  # line = '不同的时间和故事让我能驾驭所有不只是这个伴奏'
  # line = '闇に落ちて落ちて落ちてゆく 霧 の中 に 生まれて'
  # line = 'きみ も 仲間 に入 れ たげる'
  # line = 'これからもこれからもぼくがずっといちばんなんだ'
  # line = 'そっとそっとなんにも知らないまんま'
  # line= 'そっと そっと なんにも 知らない まんま'
  # line='全部知ってるから'
  # line= '二一 ゼロ'
  # line='あなぐら ぐ ら し の 渦 の 中'
  # line = 'きみ と僕と は い つ まで も'
  # line = '有象 に無 象 の 魑魅魍魎 さあ 墓場 で 踊りましょう'
  # line = '怨念 が怨 念 が怨 念 が 陽気で'
  # line = '有象 に無 象 の 魑魅魍魎 さあ 墓場 で 踊りましょう チャチャウッ'
  # line = 'キライな自分に刑事罰ぐるぐ る 思 考 の解 放'
  # line = 'きみ も 仲間 に入 れ たげる'
  # line='とり憑いたとり 憑 い た と り 憑いた 何か が'
  # line = '黄泉 の 国 へ のみ ち づれ に'
  # line = 'らららら らん らららら らん 踊り 踊り 踊ろうよ'
  # line = 'ど こにも い けない生まれた とき から'
  # line= 'ど こにも い けない生まれた とき から'
  # line = 'こ れからさ これから さ 会える きみ と 約束 する よ'
  # line = 'こ れからも こ れからも ぼ くが ずっと いちばんな んだ'
  # line = 'ああ神様子どもの神様'
  # line = '目がふらふら音もふにゃふにゃ'

  line = line.replace(' ','')
  j,k = translating(line)

  trans, pron = j,k.translate(str.maketrans("","",p2)).translate(str.maketrans("","",p1)).replace('-','')

  liner_main = pron.lower().split()[::-1]
  debug = 1
  if debug==1:
    print(line)
    print(liner_main[::-1])
    print('\n')

  w=''
  character = ''
  add_to = []
  word_gotten = 0

  for character in line[::-1]:
    w=character+w
    
    t, p = translating('('+line+')'+sep+w)
    p = p.lower().split(sep.replace(' ',''))[-1].lstrip('-.,').strip().translate(str.maketrans("","",p2)).translate(str.maketrans("","",p1)).replace('-','')
    
    liner = liner_main[word_gotten:word_gotten+1]
    if debug ==1:
      # print('w [',w,'] character [',character,'] p [',p,'] liner',liner,'gotten [',word_gotten,']')
      print('[',w,'] <-- [',character,']                  [',p,'] <--> ',liner,' [',word_gotten,']')
      # print('add_to',add_to)
      
      

    if p in liner:
      if debug==1:
        print('------type 1----------natural match---------')
        
      add_to = [w]+add_to
      w=''
      word_gotten+=1
      print(add_to,'[',word_gotten,']\n')
      continue
    
    
    if ''.join(p.split()) in liner:
      if debug==1:
        print('------type 2----------separately match---------')
      add_to =[w]+add_to
      w=''
      word_gotten+=1
      print(add_to,'[',word_gotten,']\n')
      continue
    
    if liner == []:
      if debug==1:
        print('------type Y----------residual trailing match---------')
      
      add_to = [w+add_to[0]]+add_to[1:]
      w=''
      print(add_to,'[',word_gotten,']\n')
      continue

    # if ''.join(p.split())[1:] == liner[0][1:]:
    #   if debug==1:
    #     print('------type 3----------partial separate match---------')
      
    #   add_to = [w]+add_to
    #   w=''
    #   word_gotten+=1
    #   print(add_to,'[',word_gotten,']\n')
    #   continue

      

    if len(p.split())>1:
    
      if len(p.split())==2 and len(w)==2 and p.split()[-1] in liner:
        if debug==1:
          print('------type 4----------second word match---------')
    
        add_to = [w[1:]]+add_to
        w=w[0]
        word_gotten+=1
        print(add_to,'[',word_gotten,']\n')
      if p.split()[0] in liner_main[word_gotten:word_gotten+1]:
        if debug==1:
          print('------type 5----------further second word match---------')

        add_to = [w]+add_to
        w=''
        word_gotten+=1
      
        print(add_to,'[',word_gotten,']\n')
        continue
      else:
        continue
    
    
  if w!= '':
    if debug==1:
      print('[',w,'] <-- [',character,']                  [',p,'] <--> ',liner)
      print('------type X----------bad match---------')

    w=w.strip()
    if len(w.split())==len(p):
      for item in w.split()[::-1]:
        add_to = [w]+add_to
        # w=''
        word_gotten+=1  
    else:
      if debug ==1:
        print('------type MEGA ULTRA PSYCHO CRAZY ULTIMATE----------bad match---------')
      line1 = w
      j1,k1 = translating(line1)
      trans1,pron1 = j1,k1.translate(str.maketrans("","",p2)).translate(str.maketrans("","",p1)).replace('-','')
      liner_main1 = pron1.lower().split()
      w1 = ''
      character1 = ''
      add_to1 = []
      word_gotten1 = 0 

      for character1 in line1:
        w1=w1+character1
        print(w1)
        t1,p11 = translating('('+line1+')'+sep+w1)
        p11 = p11.lower().split(sep.replace(' ',''))[-1].lstrip('-.,').strip().translate(str.maketrans("","",p2)).translate(str.maketrans("","",p1)).replace('-','')
        # print(p11)
        liner1 = liner_main1[word_gotten1:word_gotten1+1]
        if debug ==1:
          # print('w [',w,'] character [',character,'] p [',p,'] liner',liner,'gotten [',word_gotten,']')
          print('[',w1,'] <-- [',character1,']                  [',p11,'] <--> ',liner1,' [',word_gotten1,']')
        if p11 in liner1:
          if debug==1:
            print('------type 1----------natural match---------')
          add_to1 += [w1]
          w1=''
          word_gotten1+=1
          print(add_to1,'[',word_gotten1,']\n')
          continue

      if w1!= '':
        add_to1 +=[w1]

      add_to = add_to1+add_to

    if debug ==1:
    # print('w [',w,'] character [',character,'] p [',p,'] liner',liner,'gotten [',word_gotten,']')
      
      print(add_to,'[',word_gotten,']\n')

  clear()
  #########################################################
  w = ''
  for item in add_to:
    w+=item.strip()+' '
  w = w.strip()
  if debug == 1:
    print('\n----words-'+str(len(w.split()))+'--total_character---'+str(len(w.replace(' ','')))+'-----\n')
    print(w.split())
    print(pron.lower().split())
    print('\n----words-'+str(len(pron.split()))+'--total_character---'+str(len(line.replace(' ','')))+'-----\n')

    

    


###################################################################################################################################################
  if len(w.replace(' ',''))!= len(line.replace(' ','')):
    clear()
    print('PROOOOOOOOOOOOOOBLEM!!!!!!!!!!!!!!!!!')
    print('OG line length:',len(line.replace(' ','')),'Output line lenght:',len(w.replace(' ','')))

    display(line)
    display(w)
    display(add_to)
    display(k.lower().split())
    # print()
    time.sleep(1)
    os.kill(os.getpid(),9)
  else:
    sk = to_json({},'translation_database','read')
    try:
      sk[w]['pronounciation']=pron
      to_json(sk,'translation_database','write')
    except KeyError:
      sk[w]={'main':w,
      'translation':j,
      'pronounciation':pron}
      to_json(sk,'translation_database','write')
  
  return w

In [ ]:
  
# line = '不同的时间和故事让我能驾驭所有不只是这个伴奏'
def cage(line):
  # line = input()

  j,k = translating(line)
  while (j,k)==(False,False):
    j,k = translating(line)
  line = zero_width_cleaner(line,'')
  words = fontaine(line)
  width = len(k.split())
  
  f = []
  for item in words:
    # print(item['word'],sep,item['font'])
    if item['font'] != default:
      
      f.append(chinese_and_japanese(item['word']))
    else:
      f.append(item['word'])
  w=''
  for i in range(len(f)):
    w+=f[i].strip()
    # print(w)
    
    if i <len(f):
      try:
        # print(f[i])
        if f[i+1][0] not in non_space_punct and f[i+1][0] not in right_side:
          if f[i][-1] not in left_side:
            w+=' '
        else:
          continue
      except IndexError:
        break
  # print(w)
  # display(' '.join(f))
  # print(k)
  return '\n'+w.strip()


def bird_cage(line,types):
  w=''
  for lines in line.split('\n'):
    if lines not in empty_list:
      
      w+=cage(lines)

  return w




In [ ]:
# for number in range(len(video_text_sync_data[0::2][4:])):

def streamlines_lines(line):
# line = "I don't want smile"
  line = zero_width_cleaner(line,1)
  c = line
  a,b = translating(line)
  main_line,trans,pron = line,a,b
  if norm(b.translate(str.maketrans("","",p2)).lower())==norm(c.translate(str.maketrans("","",p2)).lower()):
    
    if norm(a.translate(str.maketrans("","",p2)).lower())==norm(b.translate(str.maketrans("","",p2)).lower())==norm(c.translate(str.maketrans("","",p2)).lower()):
      trans = ''
      pron = c
      main_line = ''
    else:
      main_line = ''
      pron = c
  elif norm(a.translate(str.maketrans("","",p2)).lower())==norm(b.translate(str.maketrans("","",p2)).lower()):
    trans = ''

  elif norm(a.translate(str.maketrans("","",p2)).lower())==norm(c.translate(str.maketrans("","",p2)).lower()):
    trans = ''

  if norm(c.translate(str.maketrans("","",p2)).lower())==norm(a.translate(str.maketrans("","",p2)).lower()):
    main_line = ''
    trans = ''
    pron = c
  # print(line)
  # print(main_line,sep,pron,sep, trans)
  # print('\n')

  return main_line,pron,trans



In [ ]:
def checktd():
  return index, len(to_json({},td,'read'))

# checktd()

In [ ]:
def lyrics_word_splitter():

  s = to_json({},sid,'read')
  if split_lyrics == 1:
  # print(s[index]['lyrics'])
    new = s[index]['lyrics']
    new = bird_cage(new,'')
    
  # print('\n\n')
  s[index]['lyrics']=new
  # s[index]['lyrics']
  to_json(s,sid,'write')

In [ ]:
def get_saved_colors(index,index_number_of_color_key,choose):
  col_dic = to_json({},'song_color_dictionary','read')
  song = to_json({},sid,'read')[index]
  if choose not in empty_list:
    i=0
    if len(song['color_dictionary'])>1:
      for item in song['color_dictionary']:
        i+=1
        print(str(i)+sep+item)
      print('\n')
      time.sleep(1)
      index_number_of_color_key = int(input('Enter index: '))-1
    else:
      index_number_of_color_key = ''
  if index_number_of_color_key in empty_list:
    index_number_of_color_key = 0
  else:
    index_number_of_color_key = int(index_number_of_color_key)

  color_key = song['color_dictionary'][index_number_of_color_key]  
  print('Taking color key '+color_key+' for\n'+song['song_name']+'\n')
  
  

  color_dictionary = {}
  for item in col_dic[color_key]:
    color_dictionary[item]=tuple(col_dic[color_key][item])
  return color_dictionary



In [ ]:
# collected = input('Link for another genius page containing data')
def alternate_lyrics(type_of_lyrics):
  term = done_dl[index]['file_name']
  term = re.sub(r"\([^)]*\)","",term).strip()
  try:
    term = term.split(' by ')[0]+' '+term.split(' by ')[1]
  except IndexError:
    term = term
  a = ['genius','manual','musixmatch']
  if type_of_lyrics not in a:
    search_url = "https://www.genius.com/search?q="+quote(term) 
    search_url1 = "https://"+a[-1]+".com/search/"+quote(term)
    print('Find Anything [genius.com]?? ',search_url+'\n')
    print('Find Anything [musixmatch.com]?? ',search_url1+'\n')
    
  

    return False
  if type_of_lyrics == 'genius':
    search_url = "https://www."+type_of_lyrics+".com/search?q="+quote(term)  
    print('Helpful link maybe???',sep,search_url,'\n\n')
    new = enter_song_data_from_genius_link(input('Get link for alternate lyrics for '+done_dl[index]['file_name']+'\n'),index,split_lyrics)

  if type_of_lyrics == 'manual':
    search_url = "https://www.genius.com/search?q="+quote(term)  
    print('Getting alternate lyrics for',done_dl[index]['file_name'])
    print('Helpful link maybe???',sep,search_url,'\n\n')
    print('Copy and put "\\n before". Go to this'+sep+'https://pinetools.com/add-text-each-line')
    new = input('\n\nManual lyrics: [include everything]\n')
    # if split_lyrics == 1:
    #   new = bird_cage(lyrics,'')
    # new = zero_width_cleaner(new)
    w=''
    for item in new.split('\\n'):
      if item not in empty_list:
        w+=zero_width_cleaner(item.strip(),1)+'\n'
    new = w.strip()
    
    if split_lyrics == 1:
      new = bird_cage(new,'')
  
  if type_of_lyrics == 'musixmatch':
    search_url = "https://www."+type_of_lyrics+".com/search/"+quote(term)
    print('Helpful link maybe???',sep,search_url,'\n\n')
    url = input('Get link for alternate lyrics for '+done_dl[index]['file_name']+'\n')
    
    headers = {
          'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'
      }
    r = requests.get(url,headers=headers)
    soup = bsp(r.content,'html.parser')
    w = ''
    for item in soup.find_all('p',{'class':'mxm-lyrics__content'}):
      w+=item.get_text()+'\n'
    new = w.strip()
    # display(new)


    if split_lyrics == 1:
      new = bird_cage(new,'')



  s = to_json({},sid,'read')
  # print(s[index]['lyrics'])
  s[index]['lyrics'] = new
  # print('\n\n')
  # s[index]['lyrics']
  to_json(s,sid,'write')
  return s[index]

def show_song_list(types):
  # types = input('1: for SID, 2 for SLSD\n')
  clear()
  if types in empty_list:
    return 'Oka bai'
  types = int(types)-1
  l = []
  typo = [sid,slsd] 
  print('Showing',typo[types]+'....')
  s = to_json({},typo[types],'read')
  for item in s:
    l.append(done_dl[item]['file_name']+sep+item)
  c = 0
  for item in sorted(l):
    c+=1
    a,b = item.split(sep)
    print(b,sep,a)

In [ ]:
def color_saver(index,color_dictionary):
  a = to_json({},sid,'read')
  song_color_dictionary = to_json({},'song_color_dictionary','read')
  color_key = timenow('')+sep+get_key()

  if color_key not in song_color_dictionary:
    song_color_dictionary[color_key]={}

  if 'color_dictionary' not in a[index]:
    a[index]['color_dictionary']=[]

  a[index]['color_dictionary'].append(color_key)
  song_color_dictionary[color_key] = color_dictionary
  to_json(a,sid,'write')
  to_json(song_color_dictionary,'song_color_dictionary','write')



In [ ]:
def google_link(query):  
  j = 'https://www.google.com/search?q='
  print('google: '+j+quote(query)+'&gl=us&hl=en'+'\n')
  # print('spotify: '+'https://open.spotify.com/search/'+quote(query)+'lyrics\n')
  # print('deezer: '+'https://www.deezer.com/search/'+quote(query)+'\n')
  # print('youtube: '+'https://music.youtube.com/search?q='+quote(query)+'\n')
  # print('soundcloud: '+'https://soundcloud.com/search?q='+quote(query))


In [ ]:
def extract_color_palette_of_image(image_file,number_of_colors):
  # image = Image.open(image_file)
  array = np.asarray(image_file)

  shape = array.shape
  array = array.reshape(np.product(shape[:2]),shape[2]).astype(float)


  kmeans = sklearn.cluster.MiniBatchKMeans(
      n_clusters = number_of_colors,
      init= 'k-means++',
      max_iter=20,
      random_state=1000,
  ).fit(array)

  cluster_center = kmeans.cluster_centers_
  vector,dist = scipy.cluster.vq.vq(array,cluster_center)
  counts,bins = np.histogram(vector,len(cluster_center))
  colors = []

  for index in np.argsort(counts)[::-1]:
    colors.append(tuple([int(cluster_center) for cluster_center in cluster_center[index]]))
  
  c = colors
  d = []
  for item in c:
    d.append([item])

# plt.imshow(d)
  return d
def get_color_palette_from_image():
  from matplotlib import pyplot as plt
  face_color = [0.149019607843137, 0.149019607843137, 0.149019607843137] 
  face_color = np.array([56,56,56])/255
  COLOR = [154/255,154/255,154/255]
  graph_color = [173/255, 176/255, 190/255]
  plt.rc('font',family='sans-serif',size=11)


  arial = matplotlib.font_manager.FontProperties(fname = m,size=16)

  plt.rc('lines',linewidth=1.6)
  plt.rcParams['text.color'] = COLOR
  plt.rcParams['axes.labelcolor'] = COLOR
  plt.rcParams['xtick.color'] = COLOR
  plt.rcParams['ytick.color'] = COLOR

  alp = placeholder_images+song_name
  image1,image2 = Image.open(alp+'_cover.jpg'),Image.open(alp+'_song_art.jpg')
  n_colors= 5
  colors1 = extract_color_palette_of_image(image1,n_colors)
  colors2 = extract_color_palette_of_image(image2,n_colors)
  
  if image1 != image2:
    col_pal = colors1+colors2
    fig, ax1 = plt.subplots(figsize=(7,7),facecolor=face_color,nrows=2,ncols=2)
    for ax2 in ax1:
      for ax in ax2:
        ax.set_facecolor(face_color)
        ax.grid(visible=True,color=(154/255,154/255,154/255),linewidth=.16)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)

        ax.spines['bottom'].set_color(COLOR)
        ax.spines['bottom'].set_linewidth(1.6)
        ax.spines['left'].set_linewidth(1.6)
        ax.spines['left'].set_color(COLOR)
        ax.spines['left'].set_position('zero')
        ax.set_yticks([])
        ax.set_xticks([])

    
    
    ax1[0,0].imshow(image1)
    ax1[0,0].set_title('Cover',fontproperties=arial)

    ax1[0,1].imshow(colors1)
    ax1[1,0].imshow(image2)
    ax1[1,0].set_title('Song Art',fontproperties=arial)

    ax1[1,1].imshow(colors2)
    arial = matplotlib.font_manager.FontProperties(fname = m,size=12)
    for j in range(n_colors):
      # print(j,sep,colors1[0][j])
    
      ax1[0,1].text(0.75,j,str(colors1[j][0]),va='center',ha='left',fontproperties=arial)
      ax1[0,1].text(-1,j,str(j+1),va='center',ha='right',fontproperties=arial)
      ax1[1,1].text(-1,j,str(n_colors+j+1),va='center',ha='right',fontproperties=arial)
      ax1[1,1].text(0.75,j,str(colors2[j][0]),va='center',ha='left',fontproperties=arial)
    arial = matplotlib.font_manager.FontProperties(fname = m,size=22)
    fig.suptitle(song['song_name']+' Palette',ha='center',va='center',fontproperties=arial,x=0.54)
  else:
    fig, ax1 = plt.subplots(figsize=(7,5),facecolor=face_color,nrows=1,ncols=2)
    for ax in ax1:
      # for ax in ax2:
      ax.set_facecolor(face_color)
      ax.grid(visible=True,color=(154/255,154/255,154/255),linewidth=.16)
      ax.spines['right'].set_visible(False)
      ax.spines['top'].set_visible(False)
      ax.spines['bottom'].set_visible(False)
      ax.spines['left'].set_visible(False)

      ax.spines['bottom'].set_color(COLOR)
      ax.spines['bottom'].set_linewidth(1.6)
      ax.spines['left'].set_linewidth(1.6)
      ax.spines['left'].set_color(COLOR)
      ax.spines['left'].set_position('zero')
      ax.set_yticks([])
      ax.set_xticks([])

    
    
    ax1[0].imshow(image1)
    ax1[0].set_title('Cover & Song Art',fontproperties=arial)

    ax1[1].imshow(colors1)
    
    arial = matplotlib.font_manager.FontProperties(fname = m,size=12)
    for j in range(n_colors):
      # print(j,sep,colors1[0][j])
    
      ax1[1].text(0.75,j,str(colors1[j][0]),va='center',ha='left',fontproperties=arial)
      ax1[1].text(-.75,j,str(j+1),va='center',ha='right',fontproperties=arial)
      

    arial = matplotlib.font_manager.FontProperties(fname = m,size=22)
    fig.suptitle(song['song_name']+' Palette',ha='center',va='center',fontproperties=arial,x=0.54)
    col_pal = colors1
  clear()
  fig.savefig('placeholder_images/'+song['song_name']+'_palette.jpg',dpi=300,bbox_inches='tight',facecolor=face_color)
  # time.sleep(1)
  # plt.close()
  fig=plt=ax1 = ''
  clear()
  image = Image.open('placeholder_images/'+song['song_name']+'_palette.jpg')
  display(image.resize(iresize(res,image,1)))
  # clear()
  time.sleep(1)
  # clear()
  

  p = []
  for item in col_pal:
    p.append(item[0])
    
  return p

def choiced_pick_of_colors(sort):
  import collections
  s = strips_words(lyrics,1,1)
  counts = collections.Counter(s)
  new_list = sorted(s, key=counts.get, reverse=True)
  sorted_list = list(dict.fromkeys(new_list))[0:len(col_pal)]
  print('Choose Color Indices for these words in sequence, separate by space:\n[enter 0 if you want the word to have other color]')
  print('\nTop',len(col_pal),'frequently used words in the song are:')
  display(sorted_list)
  col_choice = input().strip()
  
  if col_choice not in empty_list:
    cc = col_choice.split()
    cc1 = []
    for item in cc:
      if item not in empty_list:
        cc1.append(item)
    col_choice = cc1
    cc = col_choice[0:len(sorted_list)]
    ccd = list(dict.fromkeys(cc))
    cccd = []
    c=''
    
    for item in cc:
      
      # print(c,item)
      try:
        if item in ccd and c!=item:
          if item not in cccd:
            cccd.append(item)
            c=item
          else:
            cccd.append(0)
        else:
          cccd.append(0)
      except TypeError:
        cccd.append(item)
        c=item
    display(cccd)
    cc2 = []
    for item in cccd:
      cc2.append(int(item)-1)
    if sort not in empty_list:
      cc2 = sorted(cc2)
    dd = []
    for item in cc2:
      if item == -1:
        dd.append('')
      else:
        dd.append(col_pal[item])
  else:
  # except Exception as e:
    dd = []
  return dd



In [ ]:

def check_lyrics_translation():
  c=0
  print('TOTAL LINES',len(lyrics.split('\n')))
  for item in lyrics.split('\n'):
    c+=1
    print(str(c)+'. '+item)
    a,b = translating(item)
    print(b,sep,a+'\n')
    

  print('TOTAL LINES',len(lyrics.split('\n')))



In [ ]:
def lyrics_solver(lyrics):
  w = ''
  j = fontaine(lyrics)
  for i in range(len(j)):
    
    # w+=j[i]['word']+'\n'

    w+=j[i]['word']
    try:
      if j[i+1]['word'][0] not in non_space_punct and j[i+1]['word'][0] not in right_side:
        if j[i]['word'][-1] not in left_side:
          w+=' '
    except IndexError:
      break

  return w


def video_folder_list(t):
  d=0
  list_of_vids = os.listdir(key_destination)
  if t not in empty_list:
    list_of_vids = sorted(list_of_vids)

  for files in list_of_vids:
    d+=getfilesize(key_destination+files,2)

  print('Folder size:',format_size(d))
  print('Average file size:',format_size(d/len(list_of_vids)))
  print('Total files:',len(list_of_vids))
  print('\n\n')
  c = 0
  for files in list_of_vids:

    print(str(c+1)+'. '+files+sep+getfilesize(key_destination+files,1))
    c+=1

In [ ]:
def searcher(main_playlist):
  # s = to_json({},sid,'read')  ###THE LYRICS FILE
  if main_playlist in empty_list:
    print('SEARCHING THE MAIN PLAYLIST DATABASE')
    s = get_sd() #####MAIN PLAYLIST
  else:
    print('SEARCHING THE SONG INFO DATABASE')
    s = to_json({},sid,'read')

  data_type = list(s['1'].keys())
  item = '1'
  c= 0
  for item in data_type:
    c+=1
    print(str(c)+'. '+item.upper().replace('_',' '))

  data_type_choice = input('\nData type index: [x: to ignore] ')
  if norm(data_type_choice).lower() not in norm('x'):
    data_type = data_type[int(data_type_choice)-1]
    clear()
    data_type_that_was_chosen = data_type.upper().replace('_',' ')
    print(data_type_that_was_chosen+' chosen')
    if main_playlist in empty_list and data_type_choice == '7':
      time.sleep(1)
      key = input('Search term: [Today\'s date: ENTER] ')
      if key in empty_list:
        key = timenow('').split()[1]+' '+str(int(timenow('').split()[0]))
    
    elif main_playlist not in empty_list and data_type_choice == '4':
      time.sleep(1)
      key = input('Search term: [Today\'s date: ENTER] ')
      if key in empty_list:
        key = timenow('').split()[1]+' '+str(int(timenow('').split()[0]))

    else:
      time.sleep(1)
      key = input('Search term: ')
    clear()
    print('Searching "'+key+'" in '+data_type_that_was_chosen+'\n')
    print
    c =0
    data = []
    if data_type != 'link' and data_type !='track_position':
      for item in s:
        if s[item][data_type] is None:
          continue
        if norm(key).lower() in norm(s[item][data_type]).lower():
          c+=1
          data.append(str(c)+". Matched #"+item+'\n"'+s[item]['song_name']+'"\n'+data_type+": "+s[item][data_type]+'\n')
    else:
      if data_type == 'track_position':  
        if '/' in key:
          for item in s:
            if key == s[item][data_type]:
              c+=1
              data.append(str(c)+". Matched #"+item+'\n"'+s[item]['song_name']+'"\n'+data_type+": "+s[item][data_type]+'\n')
        else:
          for item in s:
            if key in s[item][data_type].split('/')[0]:
              c+=1
              data.append(str(c)+". Matched #"+item+'\n"'+s[item]['song_name']+'"\n'+data_type+": "+s[item][data_type]+'\n')
      else:
        for item in s:
          for provider in s[item][data_type]:
            if norm(key).lower() in norm(provider).lower():
              c+=1
              data.append(str(c)+". Matched #"+item+'\n"'+s[item]['song_name']+'"\n'+data_type+": ["+provider+"] "+s[item][data_type][provider]+'\n')
    print('\nTotal match:',len(data),'\n')
    for item in data:
      
      print(item)        
  else:
    clear()

In [ ]:
def beep():
  from google.colab import output
  return output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

def timenow(part):
  if part in empty_list:
    return datetime.now(pytz.timezone("Etc/GMT-6")).strftime('%d %B %Y, %I:%M:%S %p')
  else:
    return datetime.now(pytz.timezone("Etc/GMT-6")).strftime('%d %B %Y')

def time_difference(current,previous):
  delta = (datetime.strptime(current,'%d %B %Y, %I:%M:%S %p') - datetime.strptime(previous,'%d %B %Y, %I:%M:%S %p'))
  secondsPassed = delta.total_seconds()
  print(format_timespan(secondsPassed))
  return secondsPassed
# video_folder_list('')

In [ ]:
timenow('')

In [ ]:
#### STARTING POINT, CTRL+F8 TO RUN PREVIOUS
done_dl = update_dl()
index,lyrics_sync,track = starting_point()
regular_files = font_file_list()
start_time = timenow('')
split_lyrics = ''
final_approval = ''  
override = ''
desparate_time = ''
cage_or_word='' 
dd=[]
res = 600
try:
  if index not in to_json({},sid,'read').keys():
    google_link(get_sd()[index]['file_name']+' lyrics')
    split_lyrics = input('For Japanese and Chinese ->: [yes: 1, no: ENTER] ')
    if split_lyrics not in empty_list:
      if split_lyrics == '1':
        split_lyrics = 1  
        cage_or_word=1
    # cage_or_word = ''   #### only for chinese
    alternate_lyrics('')
    gang = input('[data in genius.com]?? ->[no: 1 yes: enter]: ')
    if gang == '1':
      desparate_time = 1
  
  if desparate_time in empty_list:
    genius_to_lyrics(index,split_lyrics,override)
  if desparate_time == 1:
    deezer_song_data_for_desparate_times()
  
  regular_files = font_file_list()
  frames = 30  
  name_of_the_song  = song_name = done_dl[index]['file_name']
  
  if lyrics_sync in empty_list and desparate_time == 1:
    method= ''
    a = ['genius','musixmatch','manual']  
    time.sleep(1)
    method = input('\nMethod of choosing alternate lyrics? ["1: genius","2: musixmatch","3: manual"]\n')
    alternate_lyrics(a[int(method)-1])
    if split_lyrics == 1 or index not in to_json({},slsd,'read').keys():
      # print('planet_translate::',index,sep,song_name)
      # awaiter = input('AWAITING PLANET TRANSLATE....')
      # break
      deezer_lyrics_splitter(lyrics_sync,split_lyrics)
      song = to_json({},sid,'read')[index]
      try:
        lyrics_word_splitter(song[index]['lyrics'])
      except KeyError:
        song = to_json({},sid,'read')[index]
  song = to_json({},sid,'read')[index]
  # lyrics = zero_width_cleaner(song['lyrics'])
  # lyrics = lyrics_from_slsd('')
  if index in to_json({},slsd,'read').keys():
    lyrics = lyrics_from_slsd('')
  else:
    if lyrics_sync not in empty_list:
      deez_lyrics,lyrics_sync = deezer_lyrics_splitter(lyrics_sync,split_lyrics)
    # lyrics = zero_width_cleaner(deez_lyrics)
  # existing_lyric_splitter()
  list_of_lyrics,total_frames,lyr_data = step_one()
  lyrics = lyr_data_to_lyrics()
  beep()
  download_song_arts()
  clear()
  
  col_pal = get_color_palette_from_image()
  final_approval = input('Confirm music video for '+song_name+'?\n[YES: [ENTER], ABORT EVERYTHING: [X]\n').strip()
  if final_approval in empty_list:
    #################################################################
    try:
      color_dictionary = get_saved_colors(index,-1,'')
    except Exception as e:
      print('Need new color for '+song_name)
      dd = choiced_pick_of_colors('')
      color_dictionary = select_color_dictionary(lyrics)             ## select one of these
    # color_dictionary = colors(lyrics)                              ##
    #################################################################
    # color_saver(index,color_dictionary)
    # lyr = makes_lyrics(res,lyrics,1)
    # card = song_card(res,lyr)
    c,t,g0,t1,g1,s_art,border_sart,video_text_sync_data = elements_for_the_mv(res)
    clear()
    song_card_v2(res)
    makes_video_v2(res,'','')
  else:
    clear()

  print('Song index:',index)
  # error.pop(index)
except KeyboardInterrupt:
  clear()
  print('oka, bai ;P')
  print('Song index:',index)
# print('\n\n')
end = time_difference(timenow(''),start_time)

In [ ]:
class SlowText(Scene):
    def construct(self):
        text = Text("This is a slowly appearing text", font_size=60)
        text.set_color_by_gradient(RED, YELLOW)
        # self.add(text)
        # self.play(Write(text, run_time=0.567))
        g= text
        fir = 0.789
        self.play(Write(g,rate_func=linear),run_time=fir,rate_func=linear,lag_ratio=0)

        self.wait(2)
        self.remove(g)
        self.wait(0.123)
        self.play(Write(g,rate_func=linear),run_time=fir,rate_func=linear,lag_ratio=0)

        self.wait(2.3)
        self.remove(g)
        self.wait(.256)
%manim -qm -r 1000,600 --disable_caching -v WARNING SlowText

In [ ]:
play_another_video('')

In [ ]:
!manim --version

In [ ]:
# get_sd()['1856']
# get_sd()['']
play_audio(1)

In [ ]:
# play_another_video('')
get_sd()['1856']

In [ ]:
l = len(get_sd())
dst = np.random.randint(0,l)
print('INdex',dst)
get_sd()[str(dst)]

In [ ]:
# index,a,b = starting_point()
song = to_json({},sid,'read')[index]
c,d = download_song_arts()
col_pal = get_color_palette_from_image()

In [ ]:
# video_folder_list('')
index,a,b=starting_point()
edit_lrc(2)

In [ ]:
# index,lyrics_sync,track = starting_point()
song = to_json({},sid,'read')[index]
lyrics = lyrics_from_slsd('').replace('\u200b','')
color_dictionary = get_saved_colors(index,-1,'')
ssm = matrix(lyrics,'',color_dictionary)
pattern = making_pattern(ssm,'x',res)
ssm

In [ ]:
# #######CURRENT DELETER
# index = '2216'
# def current_deleter(FILE):
#   # FILE = sid 
#   h = to_json({},FILE,'read')
#   h[index]
#   h.pop(index)
#   to_json(h,FILE,'write')           #####danger!!!!!!!!!!!!!!!!!!!!!!!!


# current_deleter(slsd)

In [ ]:
# # ####BULK SONG MAKER
# clone = 0

# s = to_json({},slsd,'read')
# sl = list(s.keys())
# res,frames = 600,30
# split_lyrics = ''
# desparate_time = ''
# cage_or_word = ''
# done_dl = update_dl()
# regular_files = font_file_list()
# tot = len(sl)
# part = np.ceil(tot/1)
# a,b = int(clone*part),int((clone+1)*part)-1
# i=0
# error = {}
# for item in sl:
#   i+=1
#   index = item
#   # if i > 2:
#   #   break

#   try:
#     clear()
#     song = to_json({},sid,'read')[index]
#     name_of_the_song = song_name = done_dl[index]['file_name']
#     if song_name.replace('\u200b','')+'.mp4' in os.listdir(key_destination):
#       print(song_name,'already saved...')
#       continue
#     start_time = timenow('')
#     print(a,sep,b)
#     print(str(i)+sep+'Making '+song_name+'....\n'+start_time+'\n\n')
#     list_of_lyrics, total_frames,lyr_data= step_one()
#     lyrics = lyr_data_to_lyrics()
#     color_dictionary = get_saved_colors(0,'')
#     download_song_arts()
#     c,t,g0,t1,g1,s_art,border_sart,video_text_sync_data = elements_for_the_mv(res)
#     song_card_v2(res)
#     makes_video_v2(res,'','')
#     end_time = timenow('')

#     print(song_name+' took ')
#     end = time_difference(end_time,start_time)
#     print('\n')  
  
#   except Exception as e:
#     error[index]={'song_name':song_name,
#                   'error':e,
#                   'index':index}
#     continue

# error